<a href="https://colab.research.google.com/github/MatchLab-Imperial/deep-learning-course/blob/master/03_Network_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Network Training

In this tutorial, we introduce different topics related to the training of deep neural networks, including data augmentation, optimization/regularization techniques, weight initializations and hyperparameters tuning.


In [ ]:
!pip install torchinfo

import copy
import random

import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils import data
import torchinfo
from torchvision import datasets, transforms

In [ ]:
# Utility function to control randomness for reproducibility
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

## Preliminary

We load the MNIST dataset and define a model for digit classification using our function `get_data_model`, which returns a fresh untrained model along with PyTorch data loaders for training and testing. The `train` function performs the training loop with a given loss function and optimizer, recording the training and validation losses across epochs. Finally, `plot_history` visualizes these losses to track convergence and compare different experiments.

In [ ]:
def get_data_model(train_transform=None, test_transform=None):
    # Transform: flatten images to 784 vector and normalize to [0,1]
    transform_default = transforms.Compose([
          transforms.ToTensor(),
          transforms.Lambda(lambda x: x.view(-1))
    ])
    train_transform = train_transform if train_transform is not None else transform_default
    test_transform = test_transform if test_transform is not None else transform_default

    # Load datasets
    train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=train_transform)
    test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=test_transform)

    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=100, shuffle=True)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=100, shuffle=False)

    # Model definition
    class SimpleMLP(nn.Module):
        def __init__(self):
            super(SimpleMLP, self).__init__()
            self.net = nn.Sequential(
                nn.Linear(784, 512),
                nn.ReLU(),
                nn.Linear(512, 512),
                nn.ReLU(),
                nn.Linear(512, 10),
            )
        def forward(self, x):
            return self.net(x)

    model = SimpleMLP()
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    return model, device, train_loader, test_loader

model, device, train_loader, test_loader = get_data_model()
print(torchinfo.summary(model, input_size=(1, 784)))

In [ ]:
def train(model, device, train_loader, criterion, optimizer, num_epochs=10, val_loader=None):
    train_losses = []
    val_losses = []
    train_accs = []
    val_accs = []

    model = model.to(device)

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        for inputs, targets in train_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)

            # Compute predictions and accuracy
            preds = torch.argmax(outputs, dim=1)
            correct += (preds == targets).sum().item()
            total += targets.size(0)

        epoch_loss = running_loss / total
        epoch_acc = correct / total
        train_losses.append(epoch_loss)
        train_accs.append(epoch_acc)

        if val_loader:
            model.eval()
            val_running_loss = 0.0
            val_correct = 0
            val_total = 0
            with torch.no_grad():
                for inputs, targets in val_loader:
                    inputs, targets = inputs.to(device), targets.to(device)
                    outputs = model(inputs)
                    loss = criterion(outputs, targets)
                    val_running_loss += loss.item() * inputs.size(0)

                    preds = torch.argmax(outputs, dim=1)
                    val_correct += (preds == targets).sum().item()
                    val_total += targets.size(0)

            val_loss = val_running_loss / val_total
            val_acc = val_correct / val_total
            val_losses.append(val_loss)
            val_accs.append(val_acc)

        print(f"Epoch [{epoch+1}/{num_epochs}] - "
              + f"Train loss: {epoch_loss:.4f}, Train acc: {epoch_acc*100:.2f}%"
              + (f", Val loss: {val_loss:.4f}, Val acc: {val_acc*100:.2f}%" if val_loader else ""))

    history = {
        'loss': train_losses,
        'accuracy': train_accs,
    }
    if val_loader:
        history['val_loss'] = val_losses
        history['val_accuracy'] = val_accs

    return history

In [ ]:
def plot_history(history, metric=None):
    if metric is not None and metric in history:
        plt.figure(figsize=(8, 5))
        fig, axes = plt.subplots(2, 1)

        # Top plot: Accuracy
        axes[0].plot(history[metric])
        if 'val_' + metric in history:
            axes[0].plot(history['val_' + metric])
            axes[0].legend(['Train', 'Val'])
        axes[0].set_title('Model ' + metric.capitalize())
        axes[0].set_ylabel(metric.capitalize())
        axes[0].set_xlabel('Epoch')
        axes[0].grid(True)

        fig.subplots_adjust(hspace=0.5)

        # Bottom plot: Loss
        axes[1].plot(history['loss'])
        if 'val_loss' in history:
            axes[1].plot(history['val_loss'])
            axes[1].legend(['Train', 'Val'])
        axes[1].set_title('Model Loss')
        axes[1].set_ylabel('Loss')
        axes[1].set_xlabel('Epoch')
        axes[1].grid(True)
    else:
        plt.figure(figsize=(8, 2))
        plt.plot(history['loss'])
        if 'val_loss' in history:
            plt.plot(history['val_loss'])
            plt.legend(['Train', 'Val'])
        plt.title('Model Loss')
        plt.ylabel('Loss')
        plt.xlabel('Epoch')
        plt.grid(True)
        plt.show()

## Optimizers

In a deep learning problem, the weights of a network are updated to minimize the loss in each optimization step. Optimizers define the specific method used to update in each timestep the weights of the network. For example, the vanilla Stochastic Gradient Descent (SGD) only uses the gradient of the weight with respect to the loss in the current timestep $t$, along with the learning rate, to update the weights. However, other optimizers use more sophisticate methods (e.g., keeping track of past gradient updates to use as momentum) to compute the weights' updates.

Since there are optimizers more prone to get stuck in local minima, the choice of the optimizer can potentially affect both the final performance and the speed of convergence.  A nice visualization of such behavior is the following animation, where you can notice some methods, i.e., Adadelta (yellow), and Rmsprop (black) converge significantly faster than SGD (red), which gets stuck in a local minimum.

![](http://ruder.io/content/images/2016/09/saddle_point_evaluation_optimizers.gif)

The animation above is from [Sebastian Ruder's blog](http://ruder.io/optimizing-gradient-descent/), who wrote an interesting article showing the formulation and properties of the different optimizers. We are not going to analyze the properties of the different optimizers, but you can read the blog post if you are interested in how optimizers work. As a rule of thumb, Adam is usually easier to tune due to the adaptive learning rate, whereas SGD with momentum [has been shown](https://arxiv.org/pdf/1712.07628.pdf) to reach better results when tuned correctly.

In this tutorial, we will follow a hands-on approach and focus mainly on how to use [optimizers in PyTorch](https://pytorch.org/docs/stable/optim.html). PyTorch requires you to create an optimizer object explicitly by passing it the model parameters and any desired hyperparameters, such as the learning rate. For example, the following code creates an Adam optimizer with default parameters:

In [ ]:
set_seed(42)

model, device, train_loader, test_loader = get_data_model()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)
history = train(model, device, train_loader, criterion, optimizer, num_epochs=10, val_loader=test_loader)

plot_history(history, "accuracy")

However, it is common to adjust optimizer parameters such as the learning rate, which controls how quickly the weights are updated during training. In PyTorch, you do this when you create the optimizer by specifying the desired settings. For example, the following code uses Adam with a custom learning rate of 0.0003:

In [ ]:
set_seed(42)

model, device, train_loader, test_loader = get_data_model()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=3e-4)
history = train(model, device, train_loader, criterion, optimizer, num_epochs=10, val_loader=test_loader)

plot_history(history, "accuracy")

## Initializers


Neural Networks have a large number of weights that need to be initialized properly. Weight initialization is a crucial step in tuning neural networks as different initial values can lead the model to reach different local minima. If your initial weights are too small, the network may have problems learning due to small/vanishing gradients. However, if they are too large, you may run into exploding gradients or you could saturate an activation in the network (e.g. a sigmoid). Hence, we need a proper initialization with not too large nor not too small weights. In [this link](https://www.deeplearning.ai/ai-notes/initialization/) you can find an explanation of the importance of weight initialization.  

The weights are usually initialized randomly by different algorithms, such as Xavier, He, or LeCun initialization, each designed to have specific properties under certain assumptions.

In PyTorch, you set the initialization explicitly by calling functions from `torch.nn.init` on the model's parameters after you create the layers. The available initialization methods in PyTorch are listed in the documentation. For example, to initialize a `Linear` layer so that the weights (`A` in the equation $y=Ax+b$) are drawn from a normal distribution (with a given standard deviation) and the biases (`b`) are set to zero, you can use the code below:

In [ ]:
layer = nn.Linear(in_features=784, out_features=512)

# Initialize weights with a normal distribution (mean=0, std=0.05)
nn.init.normal_(layer.weight, mean=0.0, std=0.05)

# Initialize biases to zero
nn.init.zeros_(layer.bias)

Now let's check the weights of the layers and see if they follow the distributions we set.

In [ ]:
# Access weights and biases
weight_tensor = layer.weight.detach().numpy()
bias_tensor = layer.bias.detach().numpy()

print(weight_tensor)

In [ ]:
# Now let's check that the mean is 0 and stddev is 0.05
print("Mean: {:.4f}\nStandard Deviation: {:.4f}".format(weight_tensor.mean(), weight_tensor.std()))

In [ ]:
# Let's print the bias now
print(bias_tensor)

By default, the kernel weights in PyTorch `nn.Linear` layers are initialized from a uniform distribution with bounds determined by the number of input units. Specifically, the weights are sampled uniformly from the range [-1 / √f_in, +1 / √f_in], where *f_in* is the number of input features. The bias is initialized uniformly in the same range. Another commonly used initializer is `kaiming_normal`, which draws weights from a normal distribution scaled for ReLU activations. In most cases, the default initialization in PyTorch works well and allows models to train properly without any changes.





## Loss

Another important step in training deep neural networks is the choice of the loss function, which depends on your specific problem. The full list of standard losses in PyTorch is available [here](https://pytorch.org/docs/stable/nn.html#loss-functions). Although in past tutorials we already used some losses, we now introduce two typical ones: cross-entropy and mean squared error, which are commonly used for classification and regression tasks, respectively.


### Classification

For classification problems, the standard loss used is the cross-entropy loss. For the binary case, the formula is $L = y\log(p) + (1-y)\log(1-p)$, where $p$ is a probability value between $[0, 1]$.
Typically, a [Sigmoid activation](https://en.wikipedia.org/wiki/Sigmoid_function) is applied in the binary case to constrain the output to be between 0 and 1.  

In PyTorch, this binary classification loss is implemented as `nn.BCELoss` if you apply the sigmoid activation yourself, or as `nn.BCEWithLogitsLoss`, which combines a sigmoid layer and the binary cross-entropy loss in a single function for numerical stability. This loss expects targets as a vector of values in $[0, 1]$ (usually either 0 or 1) for each input element.  

Below, we show an example of binary classification in PyTorch. To do so, we transform MNIST into a binary classification problem by classifying whether a digit is between 0–4 or 5–9.


In [ ]:
# Change to single output neuron
model, device, train_loader, test_loader = get_data_model()
model.net[-1] = nn.Linear(512, 1)
print(torchinfo.summary(model, input_size=(1, 784)))

In [ ]:
# Reload MNIST datasets with transforms
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Lambda(lambda x: x.view(-1))
])

train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

# Convert targets to binary: 0 if digit < 5, 1 if >= 5
train_dataset.targets = (train_dataset.targets >= 5).long()
test_dataset.targets = (test_dataset.targets >= 5).long()

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False)

criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

def train_binary(model, device, train_loader, criterion, optimizer, num_epochs=3, val_loader=None):
    train_losses = []
    val_losses = []
    train_accs = []
    val_accs = []

    model = model.to(device)

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0
        for inputs, targets in train_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            optimizer.zero_grad()
            outputs = model(inputs).squeeze(1)
            loss = criterion(outputs, targets.float())
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)

            preds = torch.sigmoid(outputs) > 0.5
            correct += (preds.int() == targets).sum().item()
            total += inputs.size(0)

        epoch_loss = running_loss / total
        epoch_acc = correct / total
        train_losses.append(epoch_loss)
        train_accs.append(epoch_acc)

        if val_loader:
            model.eval()
            val_running_loss = 0.0
            val_correct = 0
            val_total = 0
            with torch.no_grad():
                for inputs, targets in val_loader:
                    inputs, targets = inputs.to(device), targets.to(device)
                    outputs = model(inputs).squeeze(1)
                    loss = criterion(outputs, targets.float())
                    val_running_loss += loss.item() * inputs.size(0)

                    preds = torch.sigmoid(outputs) > 0.5
                    val_correct += (preds.int() == targets).sum().item()
                    val_total += inputs.size(0)

            val_loss = val_running_loss / val_total
            val_acc = val_correct / val_total
            val_losses.append(val_loss)
            val_accs.append(val_acc)

        print(f"Epoch [{epoch+1}/{num_epochs}] - Train loss: {epoch_loss:.4f}, Train acc: {epoch_acc*100:.2f}%"
              + (f", Val loss: {val_loss:.4f}, Val acc: {val_acc*100:.2f}%" if val_loader else ""))

    history = {
        'loss': train_losses,
        'accuracy': train_accs
    }
    if val_loader:
        history['val_loss'] = val_losses
        history['val_accuracy'] = val_accs

    return history

In [ ]:
set_seed(42)

history = train_binary(model, device, train_loader, criterion, optimizer, num_epochs=3,
                       val_loader=test_loader)
plot_history(history, 'accuracy')

When the number of classes is higher than 2, we use the cross-entropy loss, which is $L = -\sum_i y_i\log(p_i)$.

n PyTorch, this loss is implemented as `nn.CrossEntropyLoss`. It expects the model outputs to be raw logits (unnormalized scores) and the targets to be integer class indices in the range
[0,𝐶), where 𝐶 is the number of classes. Unlike some other frameworks, you do not need to one-hot encode the labels manually, as the function handles this internally.

Let’s see an example using the MNIST data.

In [ ]:
# Load MNIST with flattening transform
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Lambda(lambda x: x.view(-1))
])

train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

# Show first 5 labels
for i in range(5):
    label = train_dataset.targets[i].item()
    print(f"Label Value: {label}")

The input for the cross-entropy needs to be a probability distribution, meaning that the sum of the elements should be $\sum_i p_i = 1$. However, in PyTorch you do not need to apply the [Softmax activation](https://en.wikipedia.org/wiki/Softmax_function) manually, because `nn.CrossEntropyLoss` automatically applies the `LogSoftmax` function internally. This means you can pass the raw, unnormalized outputs (logits) of your model directly to the loss function and use the predefined training utilities without any extra steps.

In [ ]:
set_seed(42)

# Load model and data (MNIST)
model, device, train_loader, test_loader = get_data_model()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
history = train(
    model,
    device,
    train_loader,
    criterion,
    optimizer,
    num_epochs=3,
    val_loader=test_loader
)

plot_history(history, 'accuracy')

### Regression
In regression problems, it is common to use as losses the Mean Squared Error (MSE) or Mean Absolute Error (MAE). We now show a quick example using the California Housing dataset, which contains a set of input features and we aim to predict the price of the house.

In [ ]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load dataset
housing = fetch_california_housing()
X = housing.data
y = housing.target

# Split train/test
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale inputs
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

# Convert to tensors
x_train_tensor = torch.from_numpy(x_train).float()
y_train_tensor = torch.from_numpy(y_train).float()
x_test_tensor = torch.from_numpy(x_test).float()
y_test_tensor = torch.from_numpy(y_test).float()

# Create DataLoaders
train_dataset = torch.utils.data.TensorDataset(x_train_tensor, y_train_tensor)
test_dataset = torch.utils.data.TensorDataset(x_test_tensor, y_test_tensor)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False)

To predict float numbers as the price of the houses, we need a network that has only one output, so we define the `Regressor`.

In [ ]:
class Regressor(nn.Module):
    def __init__(self):
        super(Regressor, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(8, 100),
            nn.ReLU(),
            nn.Linear(100, 1)
        )

    def forward(self, x):
        return self.net(x).squeeze(1)

In [ ]:
def train_regression(model, device, train_loader, criterion, optimizer, num_epochs=100, val_loader=None):
    train_mae = []
    val_mae = []
    train_mse = []
    val_mse = []

    model = model.to(device)

    mae_criterion = nn.L1Loss()
    mse_criterion = nn.MSELoss()

    for epoch in range(num_epochs):
        model.train()
        running_mae = 0.0
        running_mse = 0.0

        for inputs, targets in train_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            mae_loss = mae_criterion(outputs, targets)
            mse_loss = mse_criterion(outputs, targets)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            running_mae += mae_loss.item() * inputs.size(0)
            running_mse += mse_loss.item() * inputs.size(0)

        epoch_mae = running_mae / len(train_loader.dataset)
        epoch_mse = running_mse / len(train_loader.dataset)
        train_mae.append(epoch_mae)
        train_mse.append(epoch_mse)

        if val_loader:
            model.eval()
            val_running_mae = 0.0
            val_running_mse = 0.0
            with torch.no_grad():
                for inputs, targets in val_loader:
                    inputs, targets = inputs.to(device), targets.to(device)
                    outputs = model(inputs)
                    mae_loss = mae_criterion(outputs, targets)
                    mse_loss = mse_criterion(outputs, targets)
                    val_running_mae += mae_loss.item() * inputs.size(0)
                    val_running_mse += mse_loss.item() * inputs.size(0)
            val_mae_epoch = val_running_mae / len(val_loader.dataset)
            val_mse_epoch = val_running_mse / len(val_loader.dataset)
            val_mae.append(val_mae_epoch)
            val_mse.append(val_mse_epoch)

        print(
            f"Epoch [{epoch+1}/{num_epochs}] - "
            f"Train MAE: {epoch_mae:.4f}, Train MSE: {epoch_mse:.4f}"
            + (
                f", Val MAE: {val_mae_epoch:.4f}, Val MSE: {val_mse_epoch:.4f}"
                if val_loader
                else ""
            )
        )

    history = {
        'mae': train_mae,
        'mse': train_mse,
    }
    if val_loader:
        history['val_mae'] = val_mae
        history['val_mse'] = val_mse

    return history

In [ ]:
set_seed(42)

model_mae = Regressor()
criterion_mae = nn.L1Loss()
optimizer_mae = optim.Adam(model_mae.parameters(), lr=3e-4)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

history_mae = train_regression(
    model_mae,
    device,
    train_loader,
    criterion_mae,
    optimizer_mae,
    num_epochs=100,
    val_loader=test_loader
)

Now we use the MSE loss:

In [ ]:
set_seed(42)

model_mse = Regressor()
criterion_mse = nn.MSELoss()
optimizer_mse = optim.Adam(model_mse.parameters(), lr=3e-4)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

history_mse = train_regression(
    model_mse,
    device,
    train_loader,
    criterion_mse,
    optimizer_mse,
    num_epochs=100,
    val_loader=test_loader
)

In [ ]:
def plot_regression_comparison(history_mae, history_mse):
    fig, axes = plt.subplots(1, 2, figsize=(12, 4))

    # Left: MAE metric comparison
    axes[0].plot(history_mae['val_mae'], label='MAE-trained')
    axes[0].plot(history_mse['val_mae'], label='MSE-trained')
    axes[0].set_title('Validation MAE')
    axes[0].set_ylabel('MAE')
    axes[0].set_xlabel('Epoch')
    axes[0].legend()
    axes[0].grid(True)

    # Right: MSE metric comparison
    axes[1].plot(history_mae['val_mse'], label='MAE-trained')
    axes[1].plot(history_mse['val_mse'], label='MSE-trained')
    axes[1].set_title('Validation MSE')
    axes[1].set_ylabel('MSE')
    axes[1].set_xlabel('Epoch')
    axes[1].legend()
    axes[1].grid(True)

    fig.tight_layout()
    plt.show()

plot_regression_comparison(history_mae, history_mse)

In [ ]:
def evaluate_model(model, device, loader):
    model = model.to(device)
    model.eval()
    mae = nn.L1Loss()
    mse = nn.MSELoss()
    running_mae = 0.0
    running_mse = 0.0
    with torch.no_grad():
        for inputs, targets in loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            running_mae += mae(outputs, targets).item() * inputs.size(0)
            running_mse += mse(outputs, targets).item() * inputs.size(0)
    mae_final = running_mae / len(loader.dataset)
    mse_final = running_mse / len(loader.dataset)
    return mae_final, mse_final

mae_mae, mse_mae = evaluate_model(model_mae, device, test_loader)
print(f"MAE-trained model achieves MAE: {mae_mae:.4f} and MSE: {mse_mae:.4f}")

mae_mse, mse_mse = evaluate_model(model_mse, device, test_loader)
print(f"MSE-trained model achieves MAE: {mae_mse:.4f} and MSE: {mse_mse:.4f}")

The resulting plots show that models trained with MAE loss often achieve lower MAE on the validation set compared to those trained with MSE loss. Similarly, models optimized with MSE loss typically yield better validation MSE.

A problem usually treated as a regression problem with deep learning approaches is depth estimation, where the depth of an image is estimated using the RGB information. Some other regression losses are also used in some settings (e.g. Huber loss or Reverse Huber loss, or the already introduced Mean Percentage Absolute Error), but MAE and MSE are widely used.

## Regularization
As mentioned in the lecture, regularization is an effective tool to fight some of the problems we may have during training, such as vanishing/exploding gradients or overfitting to the training set. We now mention some of the usual ways to regularize our training process.

### Loss Regularizers

[Regularization](https://pytorch.org/docs/stable/nn.html#weight-norm) introduces penalties during optimization to discourage overly complex models. By penalizing large weight values, regularization helps constrain the model’s capacity and can reduce overfitting.

In PyTorch, regularization is usually applied in two main ways:

- **Weight decay**: adds an L2 penalty on the weights directly in the optimizer. This is the most common approach and is equivalent to L2 regularization.
- **Manual penalties**: you can compute additional penalties (e.g., L1 or custom norms) on weights or activations and add them to your loss function manually.

If we define the output of a layer as $y = Wx + b$, then:

- **Weight decay (L2 penalty)**: encourages smaller $W$ and $b$.
- **Custom penalties**: you can compute L1 or L2 norms of the parameters or activations and combine them with your loss.

Standard penalties commonly used are L1, L2, and combinations of both.

In the example below, we’ll see how to compare training and validation metrics by varying the regularization strategy (e.g., different values of weight decay).

In [ ]:
# Function to train and return final training and test accuracy
def train_with_weight_decay(weight_decay, num_epochs=10):
    model, device, train_loader, test_loader = get_data_model()
    model = model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=weight_decay)

    for epoch in range(num_epochs):
        model.train()
        for inputs, targets in train_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

    # Evaluate training accuracy
    model.eval()
    correct_train = 0
    total_train = 0
    with torch.no_grad():
        for inputs, targets in train_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            predicted = torch.argmax(outputs, dim=1)
            correct_train += (predicted == targets).sum().item()
            total_train += targets.size(0)
    train_acc = correct_train / total_train

    # Evaluate test accuracy
    correct_test = 0
    total_test = 0
    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            predicted = torch.argmax(outputs, 1)
            correct_test += (predicted == targets).sum().item()
            total_test += targets.size(0)
    test_acc = correct_test / total_test

    return train_acc, test_acc

In [ ]:
set_seed(42)

reg_values = [0.001, 0.0]
train_accuracy = []
test_accuracy = []

for reg_val in reg_values:
    print(f"Training with weight decay = {reg_val}")
    train_acc, test_acc = train_with_weight_decay(weight_decay=reg_val)
    train_accuracy.append(train_acc)
    test_accuracy.append(test_acc)

In [ ]:
plt.figure()
plt.plot(reg_values, train_accuracy, marker='o')
plt.plot(reg_values, test_accuracy, marker='o')
plt.legend(['Training acc.', 'Test acc.'])
plt.title('Accuracy vs. Weight Decay (L2 regularization)')
plt.ylabel('Accuracy')
plt.xlabel('Weight Decay')
plt.show()

In this example, we show how a large regularization value dropped the final test accuracy, and also reduced the gap between training and test accuracy. The reason is that large regularization values put a constraint on the possible weights learnt by the network, which reduces the model capacity. Hence, you need to be careful when using regularization techniques as a large value can actually hurt the performance of the network.

### Dropout

Dropout is another regularization technique commonly applied to neural networks. A dropout layer randomly deactivates some neurons during training, which prevents the model from relying too heavily on specific patterns and improves generalization.

The dropout probability (i.e., the chance of disabling each unit) is a parameter you set when defining the layer. In evaluation mode, dropout is automatically disabled and has no effect on predictions.

In PyTorch, `nn.Dropout` scales the active units during training by $1/(1-p_{drop})$ so that the expected activation magnitude remains consistent between training and evaluation.

Below, we show an example where the dropout probability is set to 0.3.


In [ ]:
prob_drop = 0.3
drop_layer = nn.Dropout(p=prob_drop)

# Random input
x = np.random.random((1, 512)).astype(np.float32)
input_x = torch.tensor(x)

# Apply dropout in "training" mode
drop_layer.train()
y = drop_layer(input_x)

print("Input (first 10 elements):")
print(x[0, :10])

print("Output (first 10 elements):")
print(y[0, :10].numpy())

We now check what percentage of elements have been set to 0, and what is the scaling value the other elements have.

In [ ]:
# Calculate drop percentage
drop_percentage = (y == 0).sum().item() / y.numel()
print(f"Drop percentage (should be close to {prob_drop:.4f}): {drop_percentage:.4f}")

# Scaling factor
nonzero_mask = y != 0
scaling = y[nonzero_mask].sum().item() / input_x[nonzero_mask].sum().item()
print(f"Scaling factor (should be ~{1/(1-prob_drop):.4f}): {scaling:.4f}")

Now we show the training curves of a model without dropout and a model with dropout. First, without dropout:

In [ ]:
model, device, train_loader, test_loader = get_data_model()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())
history = train(model, device, train_loader, criterion, optimizer, num_epochs=10, val_loader=test_loader)

Now we add Dropout and define `get_data_model_dropout`.

In [ ]:
def get_data_model_dropout():
    # Same transform
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Lambda(lambda x: x.view(-1))
    ])

    train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
    test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=100, shuffle=True)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=100, shuffle=False)

    class MLPWithDropout(nn.Module):
        def __init__(self):
            super(MLPWithDropout, self).__init__()
            self.net = nn.Sequential(
                nn.Linear(784, 512),
                nn.ReLU(),
                nn.Dropout(0.3),
                nn.Linear(512, 512),
                nn.ReLU(),
                nn.Dropout(0.3),
                nn.Linear(512, 10)
            )
        def forward(self, x):
            return self.net(x)

    model = MLPWithDropout()
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    return model, device, train_loader, test_loader

In [ ]:
model_dropout, device, train_loader, test_loader = get_data_model_dropout()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model_dropout.parameters())
history_dropout = train(
    model_dropout,
    device,
    train_loader,
    criterion,
    optimizer,
    num_epochs=10,
    val_loader=test_loader
)

In [ ]:
print("Default model:\n")
plot_history(history)
print("\nModel with dropout:\n")
plot_history(history_dropout)

Notice how the validation loss stays closer to the training loss in the model with dropout compared to the original model.

## Normalization

### Batch Normalization

Batch Normalization normalizes the features by centering them around zero and rescaling them to have variance 1, which leads to faster and more stable training. To do so, batch normalization uses the statistics of the batch. Specifically, a batch normalization layer computes the mean and the standard deviation per channel, i.e. given a feature map of dimensionality $B\times H\times W\times C$ ($B$ is batch size, $H,W$ spatial dimensions and $C$ number of feature channels) the layer computes the mean $\mu$ and standard deviation $\sigma$ of the channels, where $\mu$ and $\sigma$ have dimensionality $1\times 1\times 1\times C$. Then, the mean $\mu$ and the standard deviation $\sigma$ of the batch are expanded again to $B\times H\times W\times C$ and used to standardize the input features to follow a distribution with mean 0 and variance 1. The layer output is then $y = \gamma\frac{x-\mu}{\sigma} + \beta$, where $\gamma$ and $\beta$ are learnable parameters that give the network more capacity to express different distributions.


Now we will generate a batch of 512x1 (a batch of 512 vectors of only 1 channel) from a uniform distribution under the $[0, 1)$ interval, resulting in mean 0.5 and variance 1/12. We will see how the batch normalization layer will scale the distribution to have mean 0 and variance 1.



In [ ]:
set_seed(42)

# Generate random input tensor (B, C, H, W) using PyTorch
input = torch.rand((512, 1, 10, 10), dtype=torch.float32)

# Create and apply BatchNorm2d
batch_norm = nn.BatchNorm2d(num_features=1)
batch_norm.train()
output = batch_norm(input)

# Print first 10 samples at channel 0, position (0, 0)
print(f"Input: {input[:10, 0, 0, 0]}")
print(f"Output: {output[:10, 0, 0, 0]}")

In [ ]:
# Input mean should be ~0.5 and var ~1/12=0.0833
print(f"Input mean: {input.mean():.4f}, var: {input.var():.4f}")
# Output mean should be ~0 and var ~1
print(f"Output mean: {output.mean():.4f}, var: {output.var():.4f}")

It is important to note that batch normalization changes behaviour in evaluation mode. During training, the layer tracks and updates the moving average of $\mu$ and $\sigma$ given all possible training batches. $\mu$ and $\sigma$ are then used to normalize the testing data without using the statistics from the testing batch.

Generally, using Batch Normalization results in faster training and easier convergence. It is quite standard to place Batch Normalization layers just before the activation function, but we can also achieve good performance by placing it after the activation function. So you can see blocks of Conv+BN+Activation or Conv+Activation+BN in different networks.

You can add your Batch Normalization layer as any other layer in your sequential model:

In [ ]:
model = nn.Sequential(
    nn.BatchNorm2d(num_features=512) # Example with 512 channels
)

### Instance Normalization

Instance Normalization normalizes each sample independently by centering its features around zero and rescaling them to have variance 1. Unlike batch normalization, which uses statistics aggregated across the whole mini-batch, instance normalization computes the mean and standard deviation separately for each sample and each channel. This makes it especially useful in tasks like style transfer, where preserving per-instance contrast and feature statistics is important.

Given a feature map of shape $B \times C \times H \times W$ ($B$ is batch size, $C$ is the number of channels), the layer computes the mean and standard deviation across the spatial dimensions for each channel within each sample. The input is then standardized using these statistics so that every channel in each sample has approximately zero mean and unit variance.

After normalization, learnable parameters $\gamma$ (scale) and $\beta$ (shift) are applied to each channel, allowing the network to reintroduce or adapt the normalized activations to any desired distribution.

To demonstrate the effect of instance normalization, we will generate a batch of $2048 \times 1$ (a batch of 2048 images of size $32 \times 32$ with only 1 channel) from a uniform distribution in the interval $[0, 1)$. This produces inputs with a mean close to 0.5 and variance around $1/12$.

We will see how the instance normalization layer transforms each sample independently to have mean 0 and variance 1.

In [ ]:
set_seed(42)

# Generate random input tensor (B, C, H, W)
input = torch.rand((2048, 1, 32, 32), dtype=torch.float32)

# Create and apply InstanceNorm2d without affine transformation
instance_norm = nn.InstanceNorm2d(num_features=1, affine=False)
instance_norm.train()

output = instance_norm(input)

# Print first 10 values at channel 0, height 0, width 0
print(f"Input: {input[:10, 0, 0, 0]}")
print(f"Output: {output[:10, 0, 0, 0]}")

In [ ]:
# Compute per-sample mean and var before and after
input_means = input.mean(dim=(2, 3)).squeeze(1).detach().cpu().numpy()
input_vars = input.var(dim=(2, 3), unbiased=False).squeeze(1).detach().cpu().numpy()

output_means = output.mean(dim=(2, 3)).squeeze(1).detach().cpu().numpy()
output_vars = output.var(dim=(2, 3), unbiased=False).squeeze(1).detach().cpu().numpy()

bin_edges_means = np.arange(-0.01, 0.55 + 0.01, 0.01)
bin_edges_stds = np.arange(0.05, 1.05 + 0.015, 0.015)
plt.figure(figsize=(12,5))

# Means
plt.subplot(1,2,1)
plt.hist(input_means, bins=bin_edges_means, label="Input Means")
plt.hist(output_means, bins=bin_edges_means, label="Output Means")
plt.axvline(0.5, color='black', linestyle='--', label='Ref: 0.5')
plt.axvline(0, color='red', linestyle='--', label='Ref: 0')
plt.title("Per-Sample Means")
plt.xlabel("Mean")
plt.ylabel("Frequency")
plt.legend()
plt.grid(True)

# Variances
plt.subplot(1,2,2)
plt.hist(input_vars, bins=bin_edges_stds, label="Input Variances")
plt.hist(output_vars, bins=bin_edges_stds, label="Output Variances")
plt.axvline(1/12, color='black', linestyle='--', label='Ref: 1/12')
plt.axvline(1, color='red', linestyle='--', label='Ref: 1')
plt.title("Per-Sample Variances")
plt.xlabel("Variance")
plt.ylabel("Frequency")
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

By the histograms above, we can see that the instance normalization layer has produced samples with per-sample means and variances tightly clustered around 0 and 1, as expected.

### Normalization Example

In [ ]:
# Data loader function
def get_mnist_loaders(batch_size=128):
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Lambda(lambda x: x.view(-1))
    ])
    train_dataset = datasets.MNIST(root="./data", train=True, download=True, transform=transform)
    test_dataset = datasets.MNIST(root="./data", train=False, download=True, transform=transform)
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    return train_loader, test_loader

train_loader, test_loader = get_mnist_loaders()

In [ ]:
def build_model(norm_type=None):
    layers = []
    layers.append(nn.Linear(784, 100))
    if norm_type == "batch":
        layers.append(nn.BatchNorm1d(100))
    elif norm_type == "instance":
        layers.append(nn.LayerNorm(100))
    layers.append(nn.Sigmoid())

    layers.append(nn.Linear(100, 100))
    if norm_type == "batch":
        layers.append(nn.BatchNorm1d(100))
    elif norm_type == "instance":
        layers.append(nn.LayerNorm(100))
    layers.append(nn.Sigmoid())

    layers.append(nn.Linear(100, 100))
    layers.append(nn.Sigmoid())

    layers.append(nn.Linear(100, 10))
    return nn.Sequential(*layers)

In [ ]:
set_seed(42)

histories = {}
for norm in ["no", "batch", "instance"]:
    print(f"Training model with {norm} normalization:")
    model = build_model(norm_type=(None if norm=="no" else norm))
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-3)
    histories[norm] = train(
        model, device, train_loader, criterion, optimizer, num_epochs=10, val_loader=test_loader
    )
    print()

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12,5))

# Accuracy
for norm, history in histories.items():
    axes[0].plot(history["val_accuracy"], label=f"{norm.capitalize()} Norm")
axes[0].set_title("Validation Accuracy")
axes[0].set_xlabel("Epoch")
axes[0].set_ylabel("Accuracy")
axes[0].legend()
axes[0].grid(True)

# Loss
for norm, history in histories.items():
    axes[1].plot(history["val_loss"], label=f"{norm.capitalize()} Norm")
axes[1].set_title("Validation Loss")
axes[1].set_xlabel("Epoch")
axes[1].set_ylabel("Loss")
axes[1].legend()
axes[1].grid(True)

plt.tight_layout()
plt.show()

In this example, we see that Batch Normalization speeds up convergence and achieves the best validation accuracy overall. Instance Normalization also accelerates training compared to no normalization but reaches a similar final performance after 10 epochs. Batch Normalization remains a common choice in feedforward networks, while Instance Normalization is often preferred in tasks like style transfer where per-sample normalization can be beneficial (see [*Instance Normalization: The Missing Ingredient for Fast Stylization*](https://arxiv.org/pdf/1607.08022) for details).


## HyperParameters Tuning

### Creating a Validation Set

When training our model, we need to decide the value of several hyperparameters, what regularization techniques we employ, or the loss used to train the model, among others. To decide these values we should not use as guidance the performance in the test set, as it may lead to overfitting to that set, and in turn to an erroneous estimate of the performance of the model in non-seen data. Hence, we use what is called a validation set, which we use to tweak the hyperparameters.

In PyTorch, there is no built-in *validation split* parameter in `DataLoader`. Instead, you usually manually split your training set into a training and validation set using tools like `torch.utils.data.random_split`. For example:   
```python
from torch.utils.data import random_split, DataLoader

# Suppose 'dataset' contains all training data
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size

train_subset, val_subset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_subset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_subset, batch_size=64, shuffle=False)
```
This splits 80% of the data for training and 20% for validation.

In these tutorials, we often reuse the official test sets of standard datasets (e.g., MNIST test set) as validation data, to ensure everyone works with the same split and can easily compare results. However, in a real project, you should keep the test set completely untouched and reserved for final evaluation only, after all hyperparameter tuning is complete.

### Early Stopping

We now know how to define a validation set and how to use it during the training process. An important step now is how to retrieve the model with the highest validation performance during training. Usually, you keep the model with the highest validation accuracy as your final model, and then, ideally, you would test this final model using an independent test set.
However, validation accuracy fluctuates between epochs, and may actually decrease after a few epochs due to overfitting. Hence, we would like to save the model with the best validation performance.

We already mentioned how to save the best-performing model in the introductory PyTorch tutorial, but let’s explain it again now that we know how to define a validation split. By default, PyTorch training loops keep updating the model for all $N$ epochs. To instead retrieve the model with the best validation performance, we can implement early stopping. Early stopping monitors a metric (for example, validation accuracy) and stops training after a specified number of epochs (patience) without improvement. Finally, we can restore the weights of the best model seen during training to avoid overfitting to later epochs.

For example:

In [ ]:
set_seed(42)

# Get data and model
model, device, train_loader, test_loader = get_data_model()
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

num_epochs = 20
patience = 4  # Stop after 4 epochs without improvement

best_model_wts = copy.deepcopy(model.state_dict())
best_val_acc = 0
epochs_no_improve = 0

train_losses = []
val_losses = []
val_accuracies = []

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, targets in train_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
    epoch_loss = running_loss / len(train_loader.dataset)
    train_losses.append(epoch_loss)

    # Validation
    model.eval()
    val_running_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            val_running_loss += loss.item() * inputs.size(0)
            preds = outputs.argmax(dim=1)
            correct += (preds == targets).sum().item()
            total += targets.size(0)
    val_loss = val_running_loss / len(test_loader.dataset)
    val_acc = correct / total
    val_losses.append(val_loss)
    val_accuracies.append(val_acc)

    print(
        f"Epoch [{epoch+1}/{num_epochs}] - "
        f"Train loss: {epoch_loss:.4f}, "
        f"Val loss: {val_loss:.4f}, "
        f"Val acc: {val_acc*100:.2f}%"
    )

    # Check for improvement
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        best_model_wts = copy.deepcopy(model.state_dict())
        epochs_no_improve = 0
    else:
        epochs_no_improve += 1

    if epochs_no_improve >= patience:
        print("Early stopping triggered.")
        break

# Load best model weights
model.load_state_dict(best_model_wts)

We see how the training process stopped after failing to improve `val_acc` during the number of epochs set by `patience`. We now check that the performance of the saved model is the same as the obtained in the best epoch in terms of validation accuracy.

In [ ]:
def evaluate_model(model, device, data_loader, criterion):
    model = model.to(device)
    model.eval()
    total_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, targets in data_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            total_loss += loss.item() * inputs.size(0)

            preds = outputs.argmax(dim=1)
            correct += (preds == targets).sum().item()
            total += targets.size(0)

    avg_loss = total_loss / total
    accuracy = correct / total

    return avg_loss, accuracy

val_loss, val_acc = evaluate_model(model, device, test_loader, criterion)
print(f"Test loss: {val_loss:.4f}, Test accuracy: {val_acc*100:.2f}%")

### Learning Rate

One of the most important parameters to tweak is the learning rate, which controls the update step performed during the backpropagation. In PyTorch, this can be adjusted dynamically during training using learning rate schedulers. Two common approaches are:

- **Step Learning Rate Scheduler**, which reduces the learning rate by a factor every fixed number of epochs.
- **Reduce on Plateau Scheduler**, which monitors a validation metric (like loss) and reduces the learning rate when the metric stops improving.

We will demonstrate both schedulers in the examples below.

For details, see the [PyTorch LR scheduler documentation](https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate).

In this example, we define a *step* learning rate scheduler (or `StepLR`) that decreases the learning rate by a factor of 0.1 every 3 epochs. This allows the optimizer to take larger steps in the beginning and finer adjustments later in training.


In [ ]:
set_seed(42)

model, device, train_loader, test_loader = get_data_model()
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# Define StepLR scheduler: decay LR by 0.1 every 3 epochs
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

step_lr_history = []
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, targets in train_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)

    epoch_train_loss = running_loss / len(train_loader.dataset)

    model.eval()
    val_running_loss = 0.0
    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            val_running_loss += loss.item() * inputs.size(0)
    epoch_val_loss = val_running_loss / len(test_loader.dataset)

    # Step the scheduler
    scheduler.step()
    current_lr = optimizer.param_groups[0]['lr']
    step_lr_history.append(current_lr)

    print(
        f"Epoch [{epoch+1}/{num_epochs}] - Train Loss: {epoch_train_loss:.4f}, Val Loss: {epoch_val_loss:.4f}, LR: {current_lr:.2e}"
    )

print(f"\nInitial Learning Rate: {step_lr_history[0]:.2e}")
print(f"Final Learning Rate: {step_lr_history[-1]:.2e}")

Another learning rate scheduler is `ReduceLROnPlateau`, which reduces the learning rate whenever a given metric has stopped improving. There are 5 important arguments:

 * `monitor`: we specify the metric we want to track
 * `patience`: number of epochs without improvement before reducing lr
 * `factor`: the new learning rate will be `new_lr = lr * factor`
 * `min_lr`: sets the minimum lr
 * `min_delta`: margin to define when the metric has stopped improving


In [ ]:
model, device, train_loader, test_loader = get_data_model()
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# Reduce LR on Plateau: monitor validation loss
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer,
    mode='min',
    factor=0.2,
    patience=1,
    threshold=0.01,
    min_lr=1e-5
)

plateau_lr_history = []
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, targets in train_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)

    epoch_train_loss = running_loss / len(train_loader.dataset)
    train_losses.append(epoch_train_loss)

    model.eval()
    val_running_loss = 0.0
    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            val_running_loss += loss.item() * inputs.size(0)
    epoch_val_loss = val_running_loss / len(test_loader.dataset)
    val_losses.append(epoch_val_loss)

    # Step the scheduler
    scheduler.step(epoch_val_loss)
    current_lr = optimizer.param_groups[0]['lr']
    plateau_lr_history.append(current_lr)

    print(
        f"Epoch [{epoch+1}/{num_epochs}] - Train Loss: {epoch_train_loss:.4f}, Val Loss: {epoch_val_loss:.4f}, LR: {current_lr:.2e}"
    )

print()
print(f"Initial Learning Rate: {plateau_lr_history[0]:.2e}")
print(f"Final Learning Rate: {plateau_lr_history[-1]:.2e}")

Again, we check how the learning rate has changed. You can check that the learning has indeed decreased when the `val_loss` has not improved by more than 0.01 until it reached the `min_lr` value.

In [ ]:
plt.figure(figsize=(8,5))
plt.plot(range(1, len(plateau_lr_history)+1), plateau_lr_history, marker='o', label="ReduceLROnPlateau")
plt.plot(range(1, len(step_lr_history)+1), step_lr_history, marker='o', label="StepLR")
plt.title("Learning Rate Schedules")
plt.xlabel("Epoch")
plt.ylabel("Learning Rate")
plt.yscale('log')
plt.legend()
plt.grid(True)
plt.show()

## Data augmentation
Deep learning models are data-hungry, tend to overfit with small training sets and its performance benefit from large training sets. A way to synthetically create more data is using data augmentation. Data augmentation aims to modify the training examples by applying transformations to the input data. Now, we will show some examples of data augmentation for images.

### Images
Data augmentation techniques such as rotation, scaling or cropping are usually applied in deep learning pipelines for vision applications. The idea is to take as input an image, apply a transformation to it, and then use it for training.

In PyTorch, data augmentation is typically performed using torchvision.transforms, which provides a wide range of transformations you can compose and apply to your datasets. These transformations can be combined in a Compose object and are applied on the fly during data loading, often leveraging multiple CPU workers to keep the GPU fed efficiently. The available transformations are listed [here](https://pytorch.org/vision/stable/transforms.html).




We first define a function `plot_data_augmentation` that we will use to show some augmented examples.



In [ ]:
def plot_data_augmentation(augmentation=None):
    # Basic transform: convert to tensor only
    base_transform = transforms.Compose([
        transforms.ToTensor()
    ])

    # If augmentation is provided, chain it after ToTensor
    if augmentation is not None:
        transform = transforms.Compose([
            transforms.ToTensor(),
            augmentation
        ])
    else:
        transform = base_transform

    # Load MNIST dataset with the transform
    dataset = datasets.MNIST(root="./data", train=True, download=True, transform=transform)
    loader = torch.utils.data.DataLoader(dataset, batch_size=5, shuffle=False)

    # Fetch a single batch
    images, labels = next(iter(loader))
    images_np = images.squeeze().numpy()

    plt.figure(figsize=(15,3))
    for i in range(5):
        plt.subplot(1,5,i+1)
        plt.imshow(images_np[i], cmap="gray")
        plt.axis("off")
    plt.show()

We will now visualize some of the transformations available in this preprocessing module. First, we plot some images without any transformations applied for comparison.

In [ ]:
plot_data_augmentation()

### Rotation
A standard transformation is to rotate the image. In PyTorch, you can use [`transforms.RandomRotation`](https://pytorch.org/vision/stable/transforms.html#torchvision.transforms.RandomRotation), which rotates the image by a random angle within a specified range.

For example, `transforms.RandomRotation(degrees=45)` rotates the image randomly between -45 and +45 degrees.

In [ ]:
plot_data_augmentation(transforms.RandomRotation(degrees=45))

### Shift

We can define a maximum range of both horizontal and vertical shifts using [`transforms.RandomAffine`](https://pytorch.org/vision/stable/transforms.html#torchvision.transforms.RandomAffine).

In PyTorch, shifting is handled by the `translate argument`, which specifies the maximum fraction for horizontal and vertical translation.

For example, `transforms.RandomAffine(degrees=0, translate=(0.2, 0.2))` randomly shifts the image horizontally and vertically by up to 20% of its size.

In [ ]:
plot_data_augmentation(transforms.RandomAffine(degrees=0, translate=(0.2, 0.2)))

### Zooming
Zooming into the image can also be achieved using [`transforms.RandomAffine`](https://pytorch.org/vision/stable/transforms.html#torchvision.transforms.RandomAffine).

Instead of a separate zoom transformation, `RandomAffine` uses the `scale` argument to specify the range of scaling factors applied randomly.

For example, `transforms.RandomAffine(degrees=0, scale=(0.8, 1.2))` will randomly zoom in or out by scaling the image between 80% and 120% of its original size.

In [ ]:
plot_data_augmentation(transforms.RandomAffine(degrees=0, scale=(0.8, 1.2)))

### Flip

We can apply random horizontal and/or vertical flips to augment the dataset. In PyTorch, the most common option is `transforms.RandomHorizontalFlip`, which flips images horizontally with a specified probability (default is 0.5). Vertical flips can be applied using `transforms.RandomVerticalFlip`. To randomly apply both flips independently, we can combine them in a `Compose` transform.

Below is an example where each flip has a 50% chance of being applied:




In [ ]:
flip_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5)
])

plot_data_augmentation(flip_transform)

### Data Normalization
Data normalization is usually applied in deep learning pipelines. A typical data normalization applied is data standardization, where we transform the input data to have `mean=0` and `std=1`. When you have multiple features coming from different distributions and different scales, normalization makes all of the features have a similar distribution, which avoids biasing the model towards using the features with larger values.

In images, the range of values is already in the same scale of [0-255], but normalizing the range of values to [0-1] can help us avoid starting training in the saturation regions of some activations, such as a sigmoid, where values of large magnitude will fall into the flat region of the activation. Data centering (mean=0) can also useful in some settings because the weight initializers used (e.g. Xavier) are derived following a ``mean=0`` assumption about the data distribution, but probably a model trained with [0-1] will usually converge too.

To perform normalization in PyTorch, you typically compute the mean and standard deviation of the training set manually, and then define a `transforms.Normalize()` transformation with these values. Once defined, this normalization transform can be applied consistently to both the training and test data, ensuring the inputs are standardized with `mean=0` and `std=1`.

In [ ]:
# Load MNIST training data
train_dataset_raw = datasets.MNIST(root="./data", train=True, download=True, transform=transforms.ToTensor())
train_loader_raw = torch.utils.data.DataLoader(train_dataset_raw, batch_size=len(train_dataset_raw))
train_data_tensor, _ = next(iter(train_loader_raw))
train_data_numpy = train_data_tensor.numpy()

# Compute mean and std
mean = train_data_numpy.mean()
std = train_data_numpy.std()

print(f"Train set original mean/std: {mean:.4f}/{std:.4f}")

# Define and apply normalization transform
normalize_transform = transforms.Normalize(mean=[mean], std=[std])
normalized_data = []

for img, _ in train_dataset_raw:
    img_norm = normalize_transform(img)
    normalized_data.append(img_norm)

normalized_tensor = torch.stack(normalized_data)
normalized_numpy = normalized_tensor.numpy()

print(f"Train set normalized mean/std: {normalized_numpy.mean():.4f}/{normalized_numpy.std():.4f}")

When using any kind of data normalization, it is important to note that we need to apply the same transformation to the test data for our model to work properly. In the given example we can use the statistics computed from the training data to standardize the test data:

In [ ]:
# Load MNIST test data
test_dataset_raw = datasets.MNIST(root="./data", train=False, download=True, transform=transforms.ToTensor())
test_loader_raw = torch.utils.data.DataLoader(train_dataset_raw, batch_size=len(test_dataset_raw))
test_data_tensor, _ = next(iter(test_loader_raw))
test_data_numpy = test_data_tensor.numpy()

print(f"Test set original mean/std: {test_data_numpy.mean():.4f}/{test_data_numpy.std():.4f}")

normalized_data = []
for img, _ in test_dataset_raw:
    img_norm = normalize_transform(img)
    normalized_data.append(img_norm)

normalized_tensor = torch.stack(normalized_data)
normalized_numpy = normalized_tensor.numpy()

print(f"Train set normalized mean/std: {normalized_numpy.mean():.4f}/{normalized_numpy.std():.4f}")

### Combining transformations
We have seen examples of individual data augmentation transformations. Next, we will show how to combine multiple transformations together and integrate them into a full training pipeline. In PyTorch, this is done by composing all the transformations into a single `transforms.Compose` sequence, which is applied to each training sample before it is passed to the model.

In [ ]:
set_seed(42)

# Dataset statistics (calculated before)
mean = 0.13066
std = 0.30811

# Defining a composed transform with normalization and augmentations
train_transform = transforms.Compose([
    transforms.RandomRotation(15),
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[mean], std=[std]),
    transforms.Lambda(lambda x: x.view(-1))
])

# For the test set, we only normalize
test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[mean], std=[std]),
    transforms.Lambda(lambda x: x.view(-1))
])

model, device, train_loader, test_loader = get_data_model(train_transform=train_transform, test_transform=test_transform)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())
history = train(model, device, train_loader, criterion, optimizer, num_epochs=10, val_loader=test_loader)

plot_history(history, "accuracy")

With this setup, data augmentation is applied on-the-fly during training each time a batch is loaded. At evaluation time, only normalization is performed.

> **Note**: In PyTorch, augmentations are applied dynamically when sampling batches, rather than as part of a model layer. This design avoids storing augmented copies and ensures fresh randomized transformations at every epoch.

In these examples, we showed practical examples of data augmentation for images. However, other modalities, such as text or audio can also benefit from data augmentation.

Keep in mind that applying a data augmentation strategy that is too aggressive may actually harm the performance in the test set, e.g., applying extreme zooms not likely to be present in the dataset or strong contrast shifts.

# Coursework

### Task 1: Tuning a Classification Model
In a machine learning problem, and especially when using a deep learning approach, finding the right set of hyperparameters, the right data augmentation strategy, or a good regularization method can make the difference between a model that performs poorly and a model with great accuracy.

For this exercise, you will be training a CNN to perform classification in CIFAR-10 (we use the official test set, which is why the variables are called `x_test` and `y_test`, as our validation set) and will analyze the impact of some of the most important elements presented in this tutorial.

Use the CNN we give in the code below, along with the given optimizer and number of training epochs as the default setting. Only modify the given CNN architecture to add Dropout or Batch Normalization layers when explicitly stated. Use 40 epochs to plot all of your curves. However, you can train for more epochs to find your best validation performance if your network has not finished training in those 40 epochs.

**Report:**
*  First, train the given default model without any data augmentation. Then define two data augmentation strategies (one more aggressive than the other) and train the model with data augmentation. Clearly state the two augmentation strategies you apply (i.e., the specific transformations). Discuss the training and validation loss curves for the two data augmentation strategies along with the original run without data augmentation. Attach in the appendix those training and validation curves. Report in a table the best validation accuracy obtained for the three runs (no data augmentation, data augmentation 1, data augmentation 2).

*  Without using any data augmentation, analyze the effect of using Dropout in the model. Carry out the same analysis for Batch Normalization. Finally, combine both. Report in the same table as in the data augmentation task the best validation accuracy for each of the three settings (baseline + Dropout, baseline + Batch Normalization, baseline + Batch Normalization + Dropout). The performance will vary depending on where the Dropout layers and Batch Normalization layers are, so state clearly where you added the layers, and what rate you used for the Dropout layers. Discuss the results.

* Using the default model/hyperparameters and no data augmentation, report the best validation accuracy when using `zeros` for the kernel initialization. Report the performance in the same table as in the dropout/batch normalization/data augmentation tasks. Discuss the results that you obtained.

*  Using the default model and no data augmentation, change the optimizer to SGD and train it with learning rates of `3e-3`, `1e-3` and `3e-4`. Report in a figure the training and validation loss for the three learning rate values and discuss the figure.

In [ ]:
# Loading CIFAR-10 train set to calculate mean and std for normalizations
x = datasets.CIFAR10(root="./data", train=True, download=True, transform=transforms.ToTensor())
loader = torch.utils.data.DataLoader(x, batch_size=len(x), shuffle=False)
data_tensor, _ = next(iter(loader))

# Compute mean and std per channel
mean = data_tensor.mean(dim=[0,2,3])
std = data_tensor.std(dim=[0,2,3])

mean = mean.tolist()
std = std.tolist()

print("Per-channel mean:", mean)
print("Per-channel std:", std)

In [ ]:
# Default: No augmentation
train_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
    # You can later add augmentations like:
    # transforms.RandomHorizontalFlip(),
    # transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)), etc.
])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

train_dataset = datasets.CIFAR10(root="./data", train=True, download=True, transform=train_transform)
test_dataset = datasets.CIFAR10(root="./data", train=False, download=True, transform=test_transform)

train_loader = data.DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2)
test_loader = data.DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=2)

print("Image shape:", np.array(train_dataset[0][0]).shape)
print("Total number of training samples:", len(train_dataset))
print("Total number of validation samples:", len(test_dataset))

In [ ]:
class CNNModel(nn.Module):
    def __init__(self, use_dropout=False, use_batchnorm=False, dropout_rate=0.5, init_mode="xavier_uniform"):
        super().__init__()

        def conv_block(in_channels, out_channels, use_bn, init, use_dropout, dropout_rate):
            conv = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1)
            if init == "zeros":
                nn.init.constant_(conv.weight, 0.0)
                nn.init.constant_(conv.bias, 0.0)
            else:
                nn.init.xavier_uniform_(conv.weight)
                nn.init.constant_(conv.bias, 0.0)
            layers = [conv]
            if use_bn:
                layers.append(nn.BatchNorm2d(out_channels))
            layers.append(nn.ReLU())
            if use_dropout:
                layers.append(nn.Dropout2d(dropout_rate))
            return nn.Sequential(*layers)


        # 4 conv blocks with ReLU and MaxPool2d(padding=1)
        self.features = nn.Sequential(
            conv_block(3, 32, use_batchnorm, init_mode, use_dropout, dropout_rate),
            nn.MaxPool2d(2,2, padding=0),

            conv_block(32, 64, use_batchnorm, init_mode, use_dropout, dropout_rate),
            nn.MaxPool2d(2,2, padding=0),

            conv_block(64, 128, use_batchnorm, init_mode, use_dropout, dropout_rate),
            nn.MaxPool2d(2,2, padding=0),

            conv_block(128, 256, use_batchnorm, init_mode, use_dropout, dropout_rate)
        )

        self.global_pool = nn.AdaptiveAvgPool2d((1,1))

        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Dropout(dropout_rate if use_dropout else 0.0),
            nn.Linear(256, 10)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.global_pool(x)
        x = self.classifier(x)
        return x

In [ ]:
def train_model(model, train_loader, test_loader, criterion, optimizer, epochs=40):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    train_losses, val_losses, train_accuracies, val_accuracies = [], [], [], []

    for epoch in range(epochs):
        model.train()
        running_loss = 0
        correct_train = 0
        total_train = 0

        for inputs, targets in train_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)

            preds = outputs.argmax(dim=1)
            correct_train += (preds == targets).sum().item()
            total_train += targets.size(0)

        train_loss = running_loss / len(train_loader.dataset)
        train_acc = correct_train / total_train
        train_losses.append(train_loss)
        train_accuracies.append(train_acc)

        model.eval()
        val_loss = 0
        correct_val = 0
        with torch.no_grad():
            for inputs, targets in test_loader:
                inputs, targets = inputs.to(device), targets.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                val_loss += loss.item() * inputs.size(0)
                preds = outputs.argmax(dim=1)
                correct_val += (preds == targets).sum().item()

        val_loss /= len(test_loader.dataset)
        val_acc = correct_val / len(test_loader.dataset)
        val_losses.append(val_loss)
        val_accuracies.append(val_acc)

        print(
            f"Epoch [{epoch+1}/{epochs}] "
            f"- Train Loss: {train_loss:.4f},  Train Acc: {train_acc*100:.2f}%, "
            f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc*100:.2f}%"
        )

    history = {
        "train_loss": train_losses,
        "train_acc": train_accuracies,
        "val_loss": val_losses,
        "val_acc": val_accuracies
    }
    return history

In [ ]:
set_seed(42)

# Use "model = CNNModel()" to get the default model
model = CNNModel(use_dropout=False, use_batchnorm=False, init_mode="xavier_uniform")
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=3e-4) # Edit optimizer and learning rate here.

history = train_model(model, train_loader, test_loader, criterion, optimizer, epochs=40)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils import data
import numpy as np
import matplotlib.pyplot as plt
import copy

def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# Loading CIFAR-10 train set to calculate mean and std for normalizations
x = datasets.CIFAR10(root="./data", train=True, download=True, transform=transforms.ToTensor())
loader = torch.utils.data.DataLoader(x, batch_size=len(x), shuffle=False)
data_tensor, _ = next(iter(loader))

# Compute mean and std per channel
mean = data_tensor.mean(dim=[0,2,3])
std = data_tensor.std(dim=[0,2,3])

mean = mean.tolist()
std = std.tolist()

print("Per-channel mean:", mean)
print("Per-channel std:", std)

# CNN Model Class
class CNNModel(nn.Module):
    def __init__(self, use_dropout=False, use_batchnorm=False, dropout_rate=0.5, init_mode="xavier_uniform"):
        super().__init__()

        def conv_block(in_channels, out_channels, use_bn, init, use_dropout, dropout_rate):
            conv = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1)
            if init == "zeros":
                nn.init.constant_(conv.weight, 0.0)
                nn.init.constant_(conv.bias, 0.0)
            else:
                nn.init.xavier_uniform_(conv.weight)
                nn.init.constant_(conv.bias, 0.0)
            layers = [conv]
            if use_bn:
                layers.append(nn.BatchNorm2d(out_channels))
            layers.append(nn.ReLU())
            if use_dropout:
                layers.append(nn.Dropout2d(dropout_rate))
            return nn.Sequential(*layers)

        # 4 conv blocks with ReLU and MaxPool2d
        self.features = nn.Sequential(
            conv_block(3, 32, use_batchnorm, init_mode, use_dropout, dropout_rate),
            nn.MaxPool2d(2, 2, padding=0),

            conv_block(32, 64, use_batchnorm, init_mode, use_dropout, dropout_rate),
            nn.MaxPool2d(2, 2, padding=0),

            conv_block(64, 128, use_batchnorm, init_mode, use_dropout, dropout_rate),
            nn.MaxPool2d(2, 2, padding=0),

            conv_block(128, 256, use_batchnorm, init_mode, use_dropout, dropout_rate)
        )

        self.global_pool = nn.AdaptiveAvgPool2d((1, 1))

        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Dropout(dropout_rate if use_dropout else 0.0),
            nn.Linear(256, 10)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.global_pool(x)
        x = self.classifier(x)
        return x

# Training Function
def train_model(model, train_loader, test_loader, criterion, optimizer, epochs=40):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    train_losses, val_losses, train_accuracies, val_accuracies = [], [], [], []

    for epoch in range(epochs):
        model.train()
        running_loss = 0
        correct_train = 0
        total_train = 0

        for inputs, targets in train_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)

            preds = outputs.argmax(dim=1)
            correct_train += (preds == targets).sum().item()
            total_train += targets.size(0)

        train_loss = running_loss / len(train_loader.dataset)
        train_acc = correct_train / total_train
        train_losses.append(train_loss)
        train_accuracies.append(train_acc)

        model.eval()
        val_loss = 0
        correct_val = 0
        with torch.no_grad():
            for inputs, targets in test_loader:
                inputs, targets = inputs.to(device), targets.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                val_loss += loss.item() * inputs.size(0)
                preds = outputs.argmax(dim=1)
                correct_val += (preds == targets).sum().item()

        val_loss /= len(test_loader.dataset)
        val_acc = correct_val / len(test_loader.dataset)
        val_losses.append(val_loss)
        val_accuracies.append(val_acc)

        print(
            f"Epoch [{epoch+1}/{epochs}] "
            f"- Train Loss: {train_loss:.4f}, Train Acc: {train_acc*100:.2f}%, "
            f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc*100:.2f}%"
        )

    history = {
        "train_loss": train_losses,
        "train_acc": train_accuracies,
        "val_loss": val_losses,
        "val_acc": val_accuracies
    }
    return history

# Plotting Function
def plot_training_curves(histories, labels, title):
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    for history, label in zip(histories, labels):
        axes[0].plot(history['train_loss'], label=f'{label} - Train')
        axes[0].plot(history['val_loss'], linestyle='--', label=f'{label} - Val')
    
    axes[0].set_xlabel('Epoch')
    axes[0].set_ylabel('Loss')
    axes[0].set_title(f'{title} - Loss')
    axes[0].legend()
    axes[0].grid(True)
    
    for history, label in zip(histories, labels):
        axes[1].plot(history['train_acc'], label=f'{label} - Train')
        axes[1].plot(history['val_acc'], linestyle='--', label=f'{label} - Val')
    
    axes[1].set_xlabel('Epoch')
    axes[1].set_ylabel('Accuracy')
    axes[1].set_title(f'{title} - Accuracy')
    axes[1].legend()
    axes[1].grid(True)
    
    plt.tight_layout()
    plt.savefig(f'{title.replace(" ", "_")}.png', dpi=300, bbox_inches='tight')
    plt.show()

# Dictionary to store all results
results = {}

# ============================================================================
# TASK 1: DATA AUGMENTATION
# ============================================================================
print("\n" + "="*80)
print("TASK 1: DATA AUGMENTATION")
print("="*80)

# 1. Baseline: No augmentation
print("\n--- Running Baseline (No Augmentation) ---")
train_transform_base = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

train_dataset = datasets.CIFAR10(root="./data", train=True, download=True, transform=train_transform_base)
test_dataset = datasets.CIFAR10(root="./data", train=False, download=True, transform=test_transform)

train_loader = data.DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2)
test_loader = data.DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=2)

set_seed(42)
model = CNNModel()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=3e-4)
history_base = train_model(model, train_loader, test_loader, criterion, optimizer, epochs=40)
results['No Augmentation'] = max(history_base['val_acc'])

# 2. Augmentation Strategy 1: Mild
print("\n--- Running Augmentation Strategy 1 (Mild) ---")
print("Transformations: RandomHorizontalFlip, RandomCrop(32, padding=4)")
train_transform_aug1 = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

train_dataset_aug1 = datasets.CIFAR10(root="./data", train=True, download=True, transform=train_transform_aug1)
train_loader_aug1 = data.DataLoader(train_dataset_aug1, batch_size=64, shuffle=True, num_workers=2)

set_seed(42)
model = CNNModel()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=3e-4)
history_aug1 = train_model(model, train_loader_aug1, test_loader, criterion, optimizer, epochs=40)
results['Data Aug 1 (Mild)'] = max(history_aug1['val_acc'])

# 3. Augmentation Strategy 2: Aggressive
print("\n--- Running Augmentation Strategy 2 (Aggressive) ---")
print("Transformations: RandomHorizontalFlip, RandomCrop(32, padding=4), RandomRotation(15), ColorJitter")
train_transform_aug2 = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomCrop(32, padding=4),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

train_dataset_aug2 = datasets.CIFAR10(root="./data", train=True, download=True, transform=train_transform_aug2)
train_loader_aug2 = data.DataLoader(train_dataset_aug2, batch_size=64, shuffle=True, num_workers=2)

set_seed(42)
model = CNNModel()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=3e-4)
history_aug2 = train_model(model, train_loader_aug2, test_loader, criterion, optimizer, epochs=40)
results['Data Aug 2 (Aggressive)'] = max(history_aug2['val_acc'])

# Plot augmentation comparison
plot_training_curves(
    [history_base, history_aug1, history_aug2],
    ['No Augmentation', 'Mild Aug', 'Aggressive Aug'],
    'Data Augmentation Comparison'
)

# ============================================================================
# TASK 2: DROPOUT AND BATCH NORMALIZATION
# ============================================================================
print("\n" + "="*80)
print("TASK 2: DROPOUT AND BATCH NORMALIZATION")
print("="*80)

# Reset to baseline data loaders (no augmentation)
train_dataset = datasets.CIFAR10(root="./data", train=True, download=True, transform=train_transform_base)
train_loader = data.DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2)

# 1. Baseline + Dropout
print("\n--- Running Baseline + Dropout (rate=0.3 after each conv block) ---")
set_seed(42)
model = CNNModel(use_dropout=True, dropout_rate=0.3)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=3e-4)
history_dropout = train_model(model, train_loader, test_loader, criterion, optimizer, epochs=40)
results['Baseline + Dropout'] = max(history_dropout['val_acc'])

# 2. Baseline + Batch Normalization
print("\n--- Running Baseline + Batch Normalization (after each conv layer) ---")
set_seed(42)
model = CNNModel(use_batchnorm=True)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=3e-4)
history_bn = train_model(model, train_loader, test_loader, criterion, optimizer, epochs=40)
results['Baseline + BatchNorm'] = max(history_bn['val_acc'])

# 3. Baseline + Batch Normalization + Dropout
print("\n--- Running Baseline + Batch Normalization + Dropout ---")
set_seed(42)
model = CNNModel(use_dropout=True, use_batchnorm=True, dropout_rate=0.3)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=3e-4)
history_bn_dropout = train_model(model, train_loader, test_loader, criterion, optimizer, epochs=40)
results['Baseline + BN + Dropout'] = max(history_bn_dropout['val_acc'])

# Plot regularization comparison
plot_training_curves(
    [history_base, history_dropout, history_bn, history_bn_dropout],
    ['Baseline', 'Dropout', 'BatchNorm', 'BN + Dropout'],
    'Regularization Comparison'
)

# ============================================================================
# TASK 3: ZERO INITIALIZATION
# ============================================================================
print("\n" + "="*80)
print("TASK 3: ZERO INITIALIZATION")
print("="*80)

print("\n--- Running with Zero Initialization ---")
set_seed(42)
model = CNNModel(init_mode="zeros")
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=3e-4)
history_zeros = train_model(model, train_loader, test_loader, criterion, optimizer, epochs=40)
results['Zero Initialization'] = max(history_zeros['val_acc'])

# ============================================================================
# TASK 4: SGD WITH DIFFERENT LEARNING RATES
# ============================================================================
print("\n" + "="*80)
print("TASK 4: SGD WITH DIFFERENT LEARNING RATES")
print("="*80)

learning_rates = [3e-3, 1e-3, 3e-4]
sgd_histories = []

for lr in learning_rates:
    print(f"\n--- Running SGD with learning rate {lr} ---")
    set_seed(42)
    model = CNNModel()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
    history = train_model(model, train_loader, test_loader, criterion, optimizer, epochs=40)
    sgd_histories.append(history)
    results[f'SGD lr={lr}'] = max(history['val_acc'])

# Plot SGD learning rate comparison
plot_training_curves(
    sgd_histories,
    [f'LR={lr}' for lr in learning_rates],
    'SGD Learning Rate Comparison'
)

# ============================================================================
# SUMMARY TABLE
# ============================================================================
print("\n" + "="*80)
print("SUMMARY TABLE - BEST VALIDATION ACCURACIES")
print("="*80)

print("\n{:<35} {:<20}".format("Configuration", "Best Val Accuracy (%)"))
print("-" * 55)
for config, acc in results.items():
    print("{:<35} {:<20.2f}".format(config, acc * 100))

# Save results to file
with open('results_summary.txt', 'w') as f:
    f.write("SUMMARY TABLE - BEST VALIDATION ACCURACIES\n")
    f.write("="*55 + "\n\n")
    f.write("{:<35} {:<20}\n".format("Configuration", "Best Val Accuracy (%)"))
    f.write("-" * 55 + "\n")
    for config, acc in results.items():
        f.write("{:<35} {:<20.2f}\n".format(config, acc * 100))

print("\nResults saved to 'results_summary.txt'")
print("All plots saved as PNG files")

In [ ]:
100%|██████████| 170M/170M [00:13<00:00, 13.0MB/s]
Per-channel mean: [0.491400808095932, 0.48215898871421814, 0.44653093814849854]
Per-channel std: [0.24703224003314972, 0.24348513782024384, 0.26158785820007324]

================================================================================
TASK 1: DATA AUGMENTATION
================================================================================

--- Running Baseline (No Augmentation) ---
Epoch [1/40] - Train Loss: 1.6750, Train Acc: 38.72%, Val Loss: 1.4267, Val Acc: 48.86%
Epoch [2/40] - Train Loss: 1.3556, Train Acc: 51.05%, Val Loss: 1.3491, Val Acc: 51.66%
Epoch [3/40] - Train Loss: 1.2098, Train Acc: 56.96%, Val Loss: 1.1581, Val Acc: 59.23%
Epoch [4/40] - Train Loss: 1.0949, Train Acc: 61.33%, Val Loss: 1.1044, Val Acc: 60.95%
Epoch [5/40] - Train Loss: 1.0131, Train Acc: 64.34%, Val Loss: 1.0535, Val Acc: 62.86%
Epoch [6/40] - Train Loss: 0.9497, Train Acc: 66.57%, Val Loss: 0.9494, Val Acc: 66.54%
Epoch [7/40] - Train Loss: 0.8965, Train Acc: 68.62%, Val Loss: 0.9210, Val Acc: 67.53%
Epoch [8/40] - Train Loss: 0.8539, Train Acc: 70.09%, Val Loss: 0.8674, Val Acc: 69.45%
Epoch [9/40] - Train Loss: 0.8089, Train Acc: 71.78%, Val Loss: 0.8867, Val Acc: 69.25%
Epoch [10/40] - Train Loss: 0.7719, Train Acc: 72.95%, Val Loss: 0.8218, Val Acc: 71.26%
Epoch [11/40] - Train Loss: 0.7332, Train Acc: 74.65%, Val Loss: 0.8231, Val Acc: 71.02%
Epoch [12/40] - Train Loss: 0.6991, Train Acc: 75.72%, Val Loss: 0.7697, Val Acc: 72.57%
Epoch [13/40] - Train Loss: 0.6638, Train Acc: 77.00%, Val Loss: 0.7269, Val Acc: 74.91%
Epoch [14/40] - Train Loss: 0.6325, Train Acc: 77.94%, Val Loss: 0.7180, Val Acc: 75.12%
Epoch [15/40] - Train Loss: 0.6060, Train Acc: 78.96%, Val Loss: 0.7522, Val Acc: 74.19%
Epoch [16/40] - Train Loss: 0.5778, Train Acc: 79.76%, Val Loss: 0.7226, Val Acc: 75.02%
Epoch [17/40] - Train Loss: 0.5545, Train Acc: 80.57%, Val Loss: 0.6830, Val Acc: 76.29%
Epoch [18/40] - Train Loss: 0.5243, Train Acc: 81.83%, Val Loss: 0.7132, Val Acc: 75.37%
Epoch [19/40] - Train Loss: 0.5030, Train Acc: 82.65%, Val Loss: 0.7035, Val Acc: 76.36%
Epoch [20/40] - Train Loss: 0.4783, Train Acc: 83.47%, Val Loss: 0.7261, Val Acc: 75.28%
Epoch [21/40] - Train Loss: 0.4588, Train Acc: 84.08%, Val Loss: 0.6661, Val Acc: 77.41%
Epoch [22/40] - Train Loss: 0.4337, Train Acc: 84.89%, Val Loss: 0.6851, Val Acc: 76.61%
Epoch [23/40] - Train Loss: 0.4103, Train Acc: 85.65%, Val Loss: 0.6401, Val Acc: 78.61%
Epoch [24/40] - Train Loss: 0.3882, Train Acc: 86.56%, Val Loss: 0.6770, Val Acc: 77.65%
Epoch [25/40] - Train Loss: 0.3743, Train Acc: 87.01%, Val Loss: 0.6625, Val Acc: 77.68%
Epoch [26/40] - Train Loss: 0.3486, Train Acc: 87.99%, Val Loss: 0.6566, Val Acc: 78.33%
Epoch [27/40] - Train Loss: 0.3319, Train Acc: 88.46%, Val Loss: 0.6883, Val Acc: 77.88%
Epoch [28/40] - Train Loss: 0.3122, Train Acc: 89.19%, Val Loss: 0.6775, Val Acc: 78.64%
Epoch [29/40] - Train Loss: 0.2922, Train Acc: 89.84%, Val Loss: 0.7080, Val Acc: 78.35%
Epoch [30/40] - Train Loss: 0.2723, Train Acc: 90.55%, Val Loss: 0.6947, Val Acc: 78.90%
Epoch [31/40] - Train Loss: 0.2604, Train Acc: 90.90%, Val Loss: 0.6816, Val Acc: 78.12%
Epoch [32/40] - Train Loss: 0.2376, Train Acc: 91.87%, Val Loss: 0.7204, Val Acc: 78.09%
Epoch [33/40] - Train Loss: 0.2204, Train Acc: 92.56%, Val Loss: 0.7119, Val Acc: 78.53%
Epoch [34/40] - Train Loss: 0.2037, Train Acc: 93.10%, Val Loss: 0.7626, Val Acc: 77.94%
Epoch [35/40] - Train Loss: 0.1905, Train Acc: 93.50%, Val Loss: 0.7793, Val Acc: 78.57%
Epoch [36/40] - Train Loss: 0.1756, Train Acc: 94.11%, Val Loss: 0.7295, Val Acc: 79.18%
Epoch [37/40] - Train Loss: 0.1590, Train Acc: 94.69%, Val Loss: 0.8148, Val Acc: 77.84%
Epoch [38/40] - Train Loss: 0.1441, Train Acc: 95.12%, Val Loss: 0.7836, Val Acc: 79.37%
Epoch [39/40] - Train Loss: 0.1399, Train Acc: 95.30%, Val Loss: 0.8046, Val Acc: 78.99%
Epoch [40/40] - Train Loss: 0.1165, Train Acc: 96.28%, Val Loss: 0.8696, Val Acc: 77.95%

--- Running Augmentation Strategy 1 (Mild) ---
Transformations: RandomHorizontalFlip, RandomCrop(32, padding=4)
Epoch [1/40] - Train Loss: 1.7634, Train Acc: 34.81%, Val Loss: 1.5071, Val Acc: 45.03%
Epoch [2/40] - Train Loss: 1.4662, Train Acc: 46.75%, Val Loss: 1.3927, Val Acc: 49.64%
Epoch [3/40] - Train Loss: 1.3154, Train Acc: 52.59%, Val Loss: 1.2422, Val Acc: 55.01%
Epoch [4/40] - Train Loss: 1.2031, Train Acc: 56.89%, Val Loss: 1.1852, Val Acc: 57.58%
Epoch [5/40] - Train Loss: 1.1231, Train Acc: 60.00%, Val Loss: 1.1039, Val Acc: 60.12%
Epoch [6/40] - Train Loss: 1.0657, Train Acc: 62.14%, Val Loss: 1.0506, Val Acc: 63.02%
Epoch [7/40] - Train Loss: 1.0165, Train Acc: 64.09%, Val Loss: 1.0312, Val Acc: 63.16%
Epoch [8/40] - Train Loss: 0.9738, Train Acc: 65.74%, Val Loss: 0.9619, Val Acc: 66.08%
Epoch [9/40] - Train Loss: 0.9369, Train Acc: 66.96%, Val Loss: 0.9614, Val Acc: 66.76%
Epoch [10/40] - Train Loss: 0.9057, Train Acc: 68.23%, Val Loss: 0.8736, Val Acc: 68.92%
Epoch [11/40] - Train Loss: 0.8718, Train Acc: 69.38%, Val Loss: 0.8783, Val Acc: 68.50%
Epoch [12/40] - Train Loss: 0.8444, Train Acc: 70.38%, Val Loss: 0.8186, Val Acc: 71.02%
Epoch [13/40] - Train Loss: 0.8153, Train Acc: 71.28%, Val Loss: 0.7890, Val Acc: 72.47%
Epoch [14/40] - Train Loss: 0.7901, Train Acc: 72.48%, Val Loss: 0.7642, Val Acc: 73.13%
Epoch [15/40] - Train Loss: 0.7623, Train Acc: 73.18%, Val Loss: 0.7944, Val Acc: 72.76%
Epoch [16/40] - Train Loss: 0.7433, Train Acc: 73.89%, Val Loss: 0.7800, Val Acc: 72.77%
Epoch [17/40] - Train Loss: 0.7202, Train Acc: 74.60%, Val Loss: 0.7166, Val Acc: 74.98%
Epoch [18/40] - Train Loss: 0.7010, Train Acc: 75.48%, Val Loss: 0.7230, Val Acc: 74.75%
Epoch [19/40] - Train Loss: 0.6816, Train Acc: 76.23%, Val Loss: 0.7321, Val Acc: 74.82%
Epoch [20/40] - Train Loss: 0.6602, Train Acc: 76.75%, Val Loss: 0.6773, Val Acc: 76.50%
Epoch [21/40] - Train Loss: 0.6426, Train Acc: 77.37%, Val Loss: 0.6795, Val Acc: 76.23%
Epoch [22/40] - Train Loss: 0.6265, Train Acc: 78.16%, Val Loss: 0.6735, Val Acc: 76.72%
Epoch [23/40] - Train Loss: 0.6151, Train Acc: 78.38%, Val Loss: 0.6413, Val Acc: 77.78%
Epoch [24/40] - Train Loss: 0.6043, Train Acc: 78.88%, Val Loss: 0.6874, Val Acc: 76.23%
Epoch [25/40] - Train Loss: 0.5866, Train Acc: 79.53%, Val Loss: 0.6402, Val Acc: 77.99%
Epoch [26/40] - Train Loss: 0.5776, Train Acc: 79.77%, Val Loss: 0.6440, Val Acc: 77.62%
Epoch [27/40] - Train Loss: 0.5597, Train Acc: 80.26%, Val Loss: 0.6139, Val Acc: 78.87%
Epoch [28/40] - Train Loss: 0.5532, Train Acc: 80.72%, Val Loss: 0.6237, Val Acc: 78.64%
Epoch [29/40] - Train Loss: 0.5405, Train Acc: 81.16%, Val Loss: 0.6031, Val Acc: 79.66%
Epoch [30/40] - Train Loss: 0.5275, Train Acc: 81.54%, Val Loss: 0.6175, Val Acc: 79.34%
Epoch [31/40] - Train Loss: 0.5219, Train Acc: 81.86%, Val Loss: 0.5730, Val Acc: 79.95%
Epoch [32/40] - Train Loss: 0.5080, Train Acc: 82.52%, Val Loss: 0.5929, Val Acc: 79.78%
Epoch [33/40] - Train Loss: 0.5027, Train Acc: 82.65%, Val Loss: 0.5765, Val Acc: 80.47%
Epoch [34/40] - Train Loss: 0.4949, Train Acc: 82.62%, Val Loss: 0.5911, Val Acc: 80.35%
Epoch [35/40] - Train Loss: 0.4795, Train Acc: 83.22%, Val Loss: 0.5842, Val Acc: 80.58%
Epoch [36/40] - Train Loss: 0.4733, Train Acc: 83.60%, Val Loss: 0.5897, Val Acc: 80.30%
Epoch [37/40] - Train Loss: 0.4608, Train Acc: 84.04%, Val Loss: 0.5579, Val Acc: 81.00%
Epoch [38/40] - Train Loss: 0.4576, Train Acc: 84.08%, Val Loss: 0.5627, Val Acc: 81.17%
Epoch [39/40] - Train Loss: 0.4473, Train Acc: 84.28%, Val Loss: 0.5557, Val Acc: 81.40%
Epoch [40/40] - Train Loss: 0.4399, Train Acc: 84.63%, Val Loss: 0.6165, Val Acc: 79.83%

--- Running Augmentation Strategy 2 (Aggressive) ---
Transformations: RandomHorizontalFlip, RandomCrop(32, padding=4), RandomRotation(15), ColorJitter
Epoch [1/40] - Train Loss: 1.8358, Train Acc: 32.07%, Val Loss: 1.5770, Val Acc: 42.41%
Epoch [2/40] - Train Loss: 1.5700, Train Acc: 42.63%, Val Loss: 1.4717, Val Acc: 46.38%
Epoch [3/40] - Train Loss: 1.4430, Train Acc: 48.06%, Val Loss: 1.3530, Val Acc: 50.93%
Epoch [4/40] - Train Loss: 1.3469, Train Acc: 51.74%, Val Loss: 1.2318, Val Acc: 56.38%
Epoch [5/40] - Train Loss: 1.2643, Train Acc: 54.89%, Val Loss: 1.2243, Val Acc: 55.67%
Epoch [6/40] - Train Loss: 1.2077, Train Acc: 57.07%, Val Loss: 1.1359, Val Acc: 59.62%
Epoch [7/40] - Train Loss: 1.1545, Train Acc: 59.21%, Val Loss: 1.1045, Val Acc: 60.83%
Epoch [8/40] - Train Loss: 1.1193, Train Acc: 60.40%, Val Loss: 1.0669, Val Acc: 62.27%
Epoch [9/40] - Train Loss: 1.0773, Train Acc: 62.03%, Val Loss: 0.9795, Val Acc: 65.44%
Epoch [10/40] - Train Loss: 1.0418, Train Acc: 63.31%, Val Loss: 0.9619, Val Acc: 66.24%
Epoch [11/40] - Train Loss: 1.0100, Train Acc: 64.44%, Val Loss: 0.9350, Val Acc: 67.20%
Epoch [12/40] - Train Loss: 0.9878, Train Acc: 65.22%, Val Loss: 0.9024, Val Acc: 68.41%
Epoch [13/40] - Train Loss: 0.9523, Train Acc: 66.32%, Val Loss: 0.8810, Val Acc: 69.89%
Epoch [14/40] - Train Loss: 0.9404, Train Acc: 66.94%, Val Loss: 0.8565, Val Acc: 70.12%
Epoch [15/40] - Train Loss: 0.9114, Train Acc: 68.27%, Val Loss: 0.8280, Val Acc: 71.08%
Epoch [16/40] - Train Loss: 0.8960, Train Acc: 68.56%, Val Loss: 0.8436, Val Acc: 70.40%
Epoch [17/40] - Train Loss: 0.8758, Train Acc: 69.39%, Val Loss: 0.8066, Val Acc: 72.25%
Epoch [18/40] - Train Loss: 0.8502, Train Acc: 70.55%, Val Loss: 0.7791, Val Acc: 73.22%
Epoch [19/40] - Train Loss: 0.8313, Train Acc: 71.02%, Val Loss: 0.8408, Val Acc: 71.35%
Epoch [20/40] - Train Loss: 0.8165, Train Acc: 71.44%, Val Loss: 0.7860, Val Acc: 72.52%
Epoch [21/40] - Train Loss: 0.8029, Train Acc: 71.95%, Val Loss: 0.7326, Val Acc: 74.44%
Epoch [22/40] - Train Loss: 0.7818, Train Acc: 72.58%, Val Loss: 0.7260, Val Acc: 74.73%
Epoch [23/40] - Train Loss: 0.7702, Train Acc: 73.17%, Val Loss: 0.7133, Val Acc: 75.27%
Epoch [24/40] - Train Loss: 0.7599, Train Acc: 73.38%, Val Loss: 0.7276, Val Acc: 75.09%
Epoch [25/40] - Train Loss: 0.7432, Train Acc: 74.18%, Val Loss: 0.6998, Val Acc: 75.50%
Epoch [26/40] - Train Loss: 0.7300, Train Acc: 74.46%, Val Loss: 0.6912, Val Acc: 76.53%
Epoch [27/40] - Train Loss: 0.7184, Train Acc: 74.90%, Val Loss: 0.6867, Val Acc: 76.10%
Epoch [28/40] - Train Loss: 0.7077, Train Acc: 75.16%, Val Loss: 0.7242, Val Acc: 75.82%
Epoch [29/40] - Train Loss: 0.7024, Train Acc: 75.57%, Val Loss: 0.6741, Val Acc: 76.91%
Epoch [30/40] - Train Loss: 0.6872, Train Acc: 75.99%, Val Loss: 0.6473, Val Acc: 77.92%
Epoch [31/40] - Train Loss: 0.6769, Train Acc: 76.40%, Val Loss: 0.6425, Val Acc: 77.41%
Epoch [32/40] - Train Loss: 0.6648, Train Acc: 76.70%, Val Loss: 0.6288, Val Acc: 78.81%
Epoch [33/40] - Train Loss: 0.6606, Train Acc: 77.20%, Val Loss: 0.6211, Val Acc: 78.48%
Epoch [34/40] - Train Loss: 0.6523, Train Acc: 77.05%, Val Loss: 0.6478, Val Acc: 77.87%
Epoch [35/40] - Train Loss: 0.6375, Train Acc: 77.94%, Val Loss: 0.6235, Val Acc: 79.21%
Epoch [36/40] - Train Loss: 0.6387, Train Acc: 77.67%, Val Loss: 0.6124, Val Acc: 79.04%
Epoch [37/40] - Train Loss: 0.6223, Train Acc: 78.59%, Val Loss: 0.5921, Val Acc: 79.80%
Epoch [38/40] - Train Loss: 0.6133, Train Acc: 78.34%, Val Loss: 0.6084, Val Acc: 79.32%
Epoch [39/40] - Train Loss: 0.6107, Train Acc: 78.70%, Val Loss: 0.5884, Val Acc: 80.05%
Epoch [40/40] - Train Loss: 0.6024, Train Acc: 79.06%, Val Loss: 0.6160, Val Acc: 79.11%

================================================================================
TASK 2: DROPOUT AND BATCH NORMALIZATION
================================================================================

--- Running Baseline + Dropout (rate=0.3 after each conv block) ---
Epoch [1/40] - Train Loss: 1.9768, Train Acc: 26.06%, Val Loss: 1.7116, Val Acc: 38.74%
Epoch [2/40] - Train Loss: 1.7274, Train Acc: 36.53%, Val Loss: 1.5072, Val Acc: 46.81%
Epoch [3/40] - Train Loss: 1.5972, Train Acc: 41.79%, Val Loss: 1.4041, Val Acc: 50.12%
Epoch [4/40] - Train Loss: 1.5121, Train Acc: 45.19%, Val Loss: 1.3158, Val Acc: 52.99%
Epoch [5/40] - Train Loss: 1.4422, Train Acc: 47.72%, Val Loss: 1.2608, Val Acc: 54.89%
Epoch [6/40] - Train Loss: 1.3805, Train Acc: 49.93%, Val Loss: 1.1934, Val Acc: 57.71%
Epoch [7/40] - Train Loss: 1.3353, Train Acc: 51.82%, Val Loss: 1.1542, Val Acc: 59.62%
Epoch [8/40] - Train Loss: 1.2949, Train Acc: 53.61%, Val Loss: 1.1215, Val Acc: 61.42%
Epoch [9/40] - Train Loss: 1.2585, Train Acc: 55.00%, Val Loss: 1.0749, Val Acc: 62.37%
Epoch [10/40] - Train Loss: 1.2268, Train Acc: 56.09%, Val Loss: 1.0374, Val Acc: 64.12%
Epoch [11/40] - Train Loss: 1.1984, Train Acc: 57.07%, Val Loss: 1.0171, Val Acc: 64.61%
Epoch [12/40] - Train Loss: 1.1660, Train Acc: 58.62%, Val Loss: 0.9891, Val Acc: 65.57%
Epoch [13/40] - Train Loss: 1.1393, Train Acc: 59.43%, Val Loss: 0.9619, Val Acc: 66.66%
Epoch [14/40] - Train Loss: 1.1225, Train Acc: 60.09%, Val Loss: 0.9323, Val Acc: 67.40%
Epoch [15/40] - Train Loss: 1.1036, Train Acc: 60.78%, Val Loss: 0.9212, Val Acc: 67.61%
Epoch [16/40] - Train Loss: 1.0817, Train Acc: 61.64%, Val Loss: 0.9050, Val Acc: 68.55%
Epoch [17/40] - Train Loss: 1.0614, Train Acc: 62.61%, Val Loss: 0.9012, Val Acc: 69.56%
Epoch [18/40] - Train Loss: 1.0495, Train Acc: 62.95%, Val Loss: 0.8796, Val Acc: 69.23%
Epoch [19/40] - Train Loss: 1.0301, Train Acc: 63.47%, Val Loss: 0.8545, Val Acc: 70.30%
Epoch [20/40] - Train Loss: 1.0186, Train Acc: 64.00%, Val Loss: 0.8443, Val Acc: 71.28%
Epoch [21/40] - Train Loss: 1.0028, Train Acc: 64.86%, Val Loss: 0.8320, Val Acc: 71.55%
Epoch [22/40] - Train Loss: 0.9942, Train Acc: 64.89%, Val Loss: 0.8235, Val Acc: 71.05%
Epoch [23/40] - Train Loss: 0.9780, Train Acc: 65.63%, Val Loss: 0.8131, Val Acc: 71.49%
Epoch [24/40] - Train Loss: 0.9639, Train Acc: 65.96%, Val Loss: 0.7994, Val Acc: 72.28%
Epoch [25/40] - Train Loss: 0.9541, Train Acc: 66.56%, Val Loss: 0.7823, Val Acc: 72.74%
Epoch [26/40] - Train Loss: 0.9445, Train Acc: 66.90%, Val Loss: 0.7757, Val Acc: 73.24%
Epoch [27/40] - Train Loss: 0.9325, Train Acc: 67.36%, Val Loss: 0.7766, Val Acc: 72.87%
Epoch [28/40] - Train Loss: 0.9240, Train Acc: 67.73%, Val Loss: 0.7798, Val Acc: 72.94%
Epoch [29/40] - Train Loss: 0.9139, Train Acc: 68.13%, Val Loss: 0.7534, Val Acc: 73.89%
Epoch [30/40] - Train Loss: 0.9014, Train Acc: 68.36%, Val Loss: 0.7423, Val Acc: 74.54%
Epoch [31/40] - Train Loss: 0.8942, Train Acc: 68.86%, Val Loss: 0.7447, Val Acc: 74.27%
Epoch [32/40] - Train Loss: 0.8886, Train Acc: 68.91%, Val Loss: 0.7326, Val Acc: 74.71%
Epoch [33/40] - Train Loss: 0.8858, Train Acc: 69.03%, Val Loss: 0.7267, Val Acc: 74.96%
Epoch [34/40] - Train Loss: 0.8740, Train Acc: 69.63%, Val Loss: 0.7228, Val Acc: 75.09%
Epoch [35/40] - Train Loss: 0.8633, Train Acc: 69.81%, Val Loss: 0.7129, Val Acc: 75.64%
Epoch [36/40] - Train Loss: 0.8564, Train Acc: 70.29%, Val Loss: 0.7032, Val Acc: 75.95%
Epoch [37/40] - Train Loss: 0.8554, Train Acc: 70.17%, Val Loss: 0.6985, Val Acc: 76.04%
Epoch [38/40] - Train Loss: 0.8428, Train Acc: 70.48%, Val Loss: 0.6995, Val Acc: 75.95%
Epoch [39/40] - Train Loss: 0.8413, Train Acc: 70.48%, Val Loss: 0.6965, Val Acc: 75.97%
Epoch [40/40] - Train Loss: 0.8258, Train Acc: 71.18%, Val Loss: 0.6846, Val Acc: 76.63%

--- Running Baseline + Batch Normalization (after each conv layer) ---
Epoch [1/40] - Train Loss: 1.2869, Train Acc: 54.47%, Val Loss: 1.1066, Val Acc: 60.56%
Epoch [2/40] - Train Loss: 0.9446, Train Acc: 66.53%, Val Loss: 0.9149, Val Acc: 67.07%
Epoch [3/40] - Train Loss: 0.8013, Train Acc: 72.08%, Val Loss: 0.8454, Val Acc: 69.94%
Epoch [4/40] - Train Loss: 0.7054, Train Acc: 75.50%, Val Loss: 0.7804, Val Acc: 72.52%
Epoch [5/40] - Train Loss: 0.6279, Train Acc: 78.20%, Val Loss: 0.7696, Val Acc: 72.66%
Epoch [6/40] - Train Loss: 0.5660, Train Acc: 80.39%, Val Loss: 0.7225, Val Acc: 74.49%
Epoch [7/40] - Train Loss: 0.5105, Train Acc: 82.39%, Val Loss: 0.9297, Val Acc: 68.79%
Epoch [8/40] - Train Loss: 0.4628, Train Acc: 84.26%, Val Loss: 0.7431, Val Acc: 73.99%
Epoch [9/40] - Train Loss: 0.4206, Train Acc: 85.44%, Val Loss: 0.7226, Val Acc: 75.39%
Epoch [10/40] - Train Loss: 0.3818, Train Acc: 86.73%, Val Loss: 0.7032, Val Acc: 76.12%
Epoch [11/40] - Train Loss: 0.3449, Train Acc: 88.31%, Val Loss: 0.7645, Val Acc: 75.17%
Epoch [12/40] - Train Loss: 0.3146, Train Acc: 89.23%, Val Loss: 0.8594, Val Acc: 73.45%
Epoch [13/40] - Train Loss: 0.2829, Train Acc: 90.51%, Val Loss: 0.8661, Val Acc: 73.16%
Epoch [14/40] - Train Loss: 0.2588, Train Acc: 91.23%, Val Loss: 0.7819, Val Acc: 75.94%
Epoch [15/40] - Train Loss: 0.2350, Train Acc: 92.04%, Val Loss: 0.8611, Val Acc: 74.55%
Epoch [16/40] - Train Loss: 0.2125, Train Acc: 92.85%, Val Loss: 1.0900, Val Acc: 70.81%
Epoch [17/40] - Train Loss: 0.1899, Train Acc: 93.75%, Val Loss: 0.8854, Val Acc: 75.39%
Epoch [18/40] - Train Loss: 0.1734, Train Acc: 94.36%, Val Loss: 0.9042, Val Acc: 74.81%
Epoch [19/40] - Train Loss: 0.1555, Train Acc: 94.85%, Val Loss: 0.9337, Val Acc: 75.10%
Epoch [20/40] - Train Loss: 0.1445, Train Acc: 95.25%, Val Loss: 1.2365, Val Acc: 69.87%
Epoch [21/40] - Train Loss: 0.1332, Train Acc: 95.67%, Val Loss: 1.3534, Val Acc: 68.33%
Epoch [22/40] - Train Loss: 0.1155, Train Acc: 96.32%, Val Loss: 0.9154, Val Acc: 75.37%
Epoch [23/40] - Train Loss: 0.1101, Train Acc: 96.49%, Val Loss: 0.9747, Val Acc: 75.78%
Epoch [24/40] - Train Loss: 0.1030, Train Acc: 96.64%, Val Loss: 0.9651, Val Acc: 75.31%
Epoch [25/40] - Train Loss: 0.0951, Train Acc: 97.02%, Val Loss: 1.1537, Val Acc: 73.75%
Epoch [26/40] - Train Loss: 0.0897, Train Acc: 97.18%, Val Loss: 1.0321, Val Acc: 75.70%
Epoch [27/40] - Train Loss: 0.0818, Train Acc: 97.46%, Val Loss: 1.1857, Val Acc: 72.34%
Epoch [28/40] - Train Loss: 0.0835, Train Acc: 97.40%, Val Loss: 1.1092, Val Acc: 74.67%
Epoch [29/40] - Train Loss: 0.0726, Train Acc: 97.70%, Val Loss: 1.4678, Val Acc: 70.40%
Epoch [30/40] - Train Loss: 0.0672, Train Acc: 97.95%, Val Loss: 1.2329, Val Acc: 72.92%
Epoch [31/40] - Train Loss: 0.0685, Train Acc: 97.91%, Val Loss: 1.2489, Val Acc: 73.11%
Epoch [32/40] - Train Loss: 0.0618, Train Acc: 98.11%, Val Loss: 1.1651, Val Acc: 74.75%
Epoch [33/40] - Train Loss: 0.0615, Train Acc: 98.10%, Val Loss: 1.2949, Val Acc: 72.88%
Epoch [34/40] - Train Loss: 0.0601, Train Acc: 98.06%, Val Loss: 1.0887, Val Acc: 75.99%
Epoch [35/40] - Train Loss: 0.0569, Train Acc: 98.18%, Val Loss: 1.4985, Val Acc: 71.34%
Epoch [36/40] - Train Loss: 0.0529, Train Acc: 98.40%, Val Loss: 1.3493, Val Acc: 72.56%
Epoch [37/40] - Train Loss: 0.0530, Train Acc: 98.38%, Val Loss: 1.3641, Val Acc: 73.60%
Epoch [38/40] - Train Loss: 0.0483, Train Acc: 98.52%, Val Loss: 1.4403, Val Acc: 72.77%
Epoch [39/40] - Train Loss: 0.0510, Train Acc: 98.37%, Val Loss: 1.2232, Val Acc: 74.86%
Epoch [40/40] - Train Loss: 0.0508, Train Acc: 98.42%, Val Loss: 1.2983, Val Acc: 75.19%

--- Running Baseline + Batch Normalization + Dropout ---
Epoch [1/40] - Train Loss: 1.9076, Train Acc: 28.62%, Val Loss: 1.6059, Val Acc: 42.75%
Epoch [2/40] - Train Loss: 1.6710, Train Acc: 38.61%, Val Loss: 1.4129, Val Acc: 49.71%
Epoch [3/40] - Train Loss: 1.5506, Train Acc: 43.55%, Val Loss: 1.2949, Val Acc: 53.79%
Epoch [4/40] - Train Loss: 1.4688, Train Acc: 46.71%, Val Loss: 1.2411, Val Acc: 55.47%
Epoch [5/40] - Train Loss: 1.4030, Train Acc: 49.06%, Val Loss: 1.1807, Val Acc: 58.14%
Epoch [6/40] - Train Loss: 1.3500, Train Acc: 51.15%, Val Loss: 1.1184, Val Acc: 60.35%
Epoch [7/40] - Train Loss: 1.3059, Train Acc: 52.97%, Val Loss: 1.0748, Val Acc: 62.06%
Epoch [8/40] - Train Loss: 1.2752, Train Acc: 54.32%, Val Loss: 1.0571, Val Acc: 62.21%
Epoch [9/40] - Train Loss: 1.2396, Train Acc: 55.55%, Val Loss: 1.0120, Val Acc: 64.42%
Epoch [10/40] - Train Loss: 1.2101, Train Acc: 56.79%, Val Loss: 0.9695, Val Acc: 65.79%
Epoch [11/40] - Train Loss: 1.1803, Train Acc: 57.92%, Val Loss: 0.9550, Val Acc: 66.84%
Epoch [12/40] - Train Loss: 1.1526, Train Acc: 58.94%, Val Loss: 0.9375, Val Acc: 67.40%
Epoch [13/40] - Train Loss: 1.1331, Train Acc: 59.65%, Val Loss: 0.9128, Val Acc: 68.41%
Epoch [14/40] - Train Loss: 1.1149, Train Acc: 60.47%, Val Loss: 0.8910, Val Acc: 68.82%
Epoch [15/40] - Train Loss: 1.0931, Train Acc: 61.28%, Val Loss: 0.8722, Val Acc: 69.45%
Epoch [16/40] - Train Loss: 1.0779, Train Acc: 61.84%, Val Loss: 0.8459, Val Acc: 70.21%
Epoch [17/40] - Train Loss: 1.0655, Train Acc: 62.47%, Val Loss: 0.8469, Val Acc: 70.70%
Epoch [18/40] - Train Loss: 1.0482, Train Acc: 63.03%, Val Loss: 0.8445, Val Acc: 70.51%
Epoch [19/40] - Train Loss: 1.0333, Train Acc: 63.50%, Val Loss: 0.8420, Val Acc: 70.19%
Epoch [20/40] - Train Loss: 1.0187, Train Acc: 64.03%, Val Loss: 0.8177, Val Acc: 71.46%
Epoch [21/40] - Train Loss: 1.0012, Train Acc: 64.83%, Val Loss: 0.7850, Val Acc: 72.62%
Epoch [22/40] - Train Loss: 0.9883, Train Acc: 65.37%, Val Loss: 0.7947, Val Acc: 72.10%
Epoch [23/40] - Train Loss: 0.9756, Train Acc: 65.59%, Val Loss: 0.7657, Val Acc: 73.14%
Epoch [24/40] - Train Loss: 0.9691, Train Acc: 65.84%, Val Loss: 0.7647, Val Acc: 73.36%
Epoch [25/40] - Train Loss: 0.9584, Train Acc: 66.28%, Val Loss: 0.7595, Val Acc: 73.54%
Epoch [26/40] - Train Loss: 0.9474, Train Acc: 66.81%, Val Loss: 0.7449, Val Acc: 74.03%
Epoch [27/40] - Train Loss: 0.9370, Train Acc: 66.99%, Val Loss: 0.7328, Val Acc: 74.54%
Epoch [28/40] - Train Loss: 0.9257, Train Acc: 67.49%, Val Loss: 0.7370, Val Acc: 74.16%
Epoch [29/40] - Train Loss: 0.9235, Train Acc: 67.64%, Val Loss: 0.7284, Val Acc: 74.78%
Epoch [30/40] - Train Loss: 0.9125, Train Acc: 67.96%, Val Loss: 0.7194, Val Acc: 75.04%
Epoch [31/40] - Train Loss: 0.9021, Train Acc: 68.46%, Val Loss: 0.7315, Val Acc: 74.41%
Epoch [32/40] - Train Loss: 0.8987, Train Acc: 68.60%, Val Loss: 0.7002, Val Acc: 75.68%
Epoch [33/40] - Train Loss: 0.8925, Train Acc: 68.71%, Val Loss: 0.7075, Val Acc: 75.22%
Epoch [34/40] - Train Loss: 0.8842, Train Acc: 69.01%, Val Loss: 0.6995, Val Acc: 75.70%
Epoch [35/40] - Train Loss: 0.8786, Train Acc: 69.21%, Val Loss: 0.6990, Val Acc: 75.98%
Epoch [36/40] - Train Loss: 0.8674, Train Acc: 69.86%, Val Loss: 0.6928, Val Acc: 75.86%
Epoch [37/40] - Train Loss: 0.8590, Train Acc: 69.91%, Val Loss: 0.6812, Val Acc: 76.36%
Epoch [38/40] - Train Loss: 0.8535, Train Acc: 70.12%, Val Loss: 0.6845, Val Acc: 76.50%
Epoch [39/40] - Train Loss: 0.8528, Train Acc: 70.36%, Val Loss: 0.6680, Val Acc: 77.14%
Epoch [40/40] - Train Loss: 0.8413, Train Acc: 70.48%, Val Loss: 0.6696, Val Acc: 76.56%

================================================================================
TASK 3: ZERO INITIALIZATION
================================================================================

--- Running with Zero Initialization ---
Epoch [1/40] - Train Loss: 2.3029, Train Acc: 9.85%, Val Loss: 2.3027, Val Acc: 10.00%
Epoch [2/40] - Train Loss: 2.3027, Train Acc: 9.80%, Val Loss: 2.3026, Val Acc: 10.00%
Epoch [3/40] - Train Loss: 2.3027, Train Acc: 9.92%, Val Loss: 2.3026, Val Acc: 10.00%
Epoch [4/40] - Train Loss: 2.3026, Train Acc: 9.74%, Val Loss: 2.3026, Val Acc: 10.00%
Epoch [5/40] - Train Loss: 2.3026, Train Acc: 9.65%, Val Loss: 2.3026, Val Acc: 10.00%
Epoch [6/40] - Train Loss: 2.3026, Train Acc: 9.69%, Val Loss: 2.3026, Val Acc: 10.00%
Epoch [7/40] - Train Loss: 2.3026, Train Acc: 9.80%, Val Loss: 2.3026, Val Acc: 10.00%
Epoch [8/40] - Train Loss: 2.3026, Train Acc: 9.60%, Val Loss: 2.3026, Val Acc: 10.00%
Epoch [9/40] - Train Loss: 2.3026, Train Acc: 9.74%, Val Loss: 2.3026, Val Acc: 10.00%
Epoch [10/40] - Train Loss: 2.3026, Train Acc: 9.60%, Val Loss: 2.3026, Val Acc: 10.00%
Epoch [11/40] - Train Loss: 2.3026, Train Acc: 9.84%, Val Loss: 2.3026, Val Acc: 10.00%
Epoch [12/40] - Train Loss: 2.3026, Train Acc: 9.85%, Val Loss: 2.3026, Val Acc: 10.00%
Epoch [13/40] - Train Loss: 2.3026, Train Acc: 9.66%, Val Loss: 2.3026, Val Acc: 10.00%
Epoch [14/40] - Train Loss: 2.3026, Train Acc: 9.58%, Val Loss: 2.3026, Val Acc: 10.00%
Epoch [15/40] - Train Loss: 2.3026, Train Acc: 9.88%, Val Loss: 2.3026, Val Acc: 10.00%
Epoch [16/40] - Train Loss: 2.3026, Train Acc: 9.74%, Val Loss: 2.3026, Val Acc: 10.00%
Epoch [17/40] - Train Loss: 2.3026, Train Acc: 9.81%, Val Loss: 2.3026, Val Acc: 10.00%
Epoch [18/40] - Train Loss: 2.3026, Train Acc: 9.67%, Val Loss: 2.3026, Val Acc: 10.00%
Epoch [19/40] - Train Loss: 2.3026, Train Acc: 9.73%, Val Loss: 2.3026, Val Acc: 10.00%
Epoch [20/40] - Train Loss: 2.3026, Train Acc: 9.72%, Val Loss: 2.3026, Val Acc: 10.00%
Epoch [21/40] - Train Loss: 2.3026, Train Acc: 9.83%, Val Loss: 2.3026, Val Acc: 10.00%
Epoch [22/40] - Train Loss: 2.3026, Train Acc: 9.69%, Val Loss: 2.3026, Val Acc: 10.00%
Epoch [23/40] - Train Loss: 2.3026, Train Acc: 9.68%, Val Loss: 2.3026, Val Acc: 10.00%
Epoch [24/40] - Train Loss: 2.3026, Train Acc: 9.77%, Val Loss: 2.3026, Val Acc: 10.00%
Epoch [25/40] - Train Loss: 2.3026, Train Acc: 9.84%, Val Loss: 2.3026, Val Acc: 10.00%
Epoch [26/40] - Train Loss: 2.3026, Train Acc: 9.52%, Val Loss: 2.3026, Val Acc: 10.00%
Epoch [27/40] - Train Loss: 2.3026, Train Acc: 9.67%, Val Loss: 2.3026, Val Acc: 10.00%
Epoch [28/40] - Train Loss: 2.3026, Train Acc: 9.81%, Val Loss: 2.3026, Val Acc: 10.00%
Epoch [29/40] - Train Loss: 2.3026, Train Acc: 9.88%, Val Loss: 2.3026, Val Acc: 10.00%
Epoch [30/40] - Train Loss: 2.3026, Train Acc: 9.58%, Val Loss: 2.3026, Val Acc: 10.00%
Epoch [31/40] - Train Loss: 2.3026, Train Acc: 9.74%, Val Loss: 2.3026, Val Acc: 10.00%
Epoch [32/40] - Train Loss: 2.3026, Train Acc: 9.70%, Val Loss: 2.3026, Val Acc: 10.00%
Epoch [33/40] - Train Loss: 2.3026, Train Acc: 9.75%, Val Loss: 2.3026, Val Acc: 10.00%
Epoch [34/40] - Train Loss: 2.3026, Train Acc: 9.80%, Val Loss: 2.3026, Val Acc: 10.00%
Epoch [35/40] - Train Loss: 2.3026, Train Acc: 9.67%, Val Loss: 2.3026, Val Acc: 10.00%
Epoch [36/40] - Train Loss: 2.3026, Train Acc: 9.84%, Val Loss: 2.3026, Val Acc: 10.00%
Epoch [37/40] - Train Loss: 2.3026, Train Acc: 9.68%, Val Loss: 2.3026, Val Acc: 10.00%
Epoch [38/40] - Train Loss: 2.3026, Train Acc: 9.90%, Val Loss: 2.3026, Val Acc: 10.00%
Epoch [39/40] - Train Loss: 2.3026, Train Acc: 9.52%, Val Loss: 2.3026, Val Acc: 10.00%
Epoch [40/40] - Train Loss: 2.3026, Train Acc: 9.78%, Val Loss: 2.3026, Val Acc: 10.00%

================================================================================
TASK 4: SGD WITH DIFFERENT LEARNING RATES
================================================================================

--- Running SGD with learning rate 0.003 ---
Epoch [1/40] - Train Loss: 2.0024, Train Acc: 24.99%, Val Loss: 1.7109, Val Acc: 36.10%
Epoch [2/40] - Train Loss: 1.6285, Train Acc: 39.56%, Val Loss: 1.5862, Val Acc: 40.95%
Epoch [3/40] - Train Loss: 1.4713, Train Acc: 46.55%, Val Loss: 1.4110, Val Acc: 48.76%
Epoch [4/40] - Train Loss: 1.3329, Train Acc: 51.87%, Val Loss: 1.3053, Val Acc: 52.52%
Epoch [5/40] - Train Loss: 1.2114, Train Acc: 56.61%, Val Loss: 1.2101, Val Acc: 55.39%
Epoch [6/40] - Train Loss: 1.1095, Train Acc: 60.58%, Val Loss: 1.0886, Val Acc: 62.22%
Epoch [7/40] - Train Loss: 1.0342, Train Acc: 63.27%, Val Loss: 1.0411, Val Acc: 63.00%
Epoch [8/40] - Train Loss: 0.9743, Train Acc: 65.58%, Val Loss: 0.9905, Val Acc: 65.01%
Epoch [9/40] - Train Loss: 0.9173, Train Acc: 67.53%, Val Loss: 0.9958, Val Acc: 64.79%
Epoch [10/40] - Train Loss: 0.8728, Train Acc: 69.29%, Val Loss: 0.8984, Val Acc: 68.29%
Epoch [11/40] - Train Loss: 0.8292, Train Acc: 70.92%, Val Loss: 0.8684, Val Acc: 69.26%
Epoch [12/40] - Train Loss: 0.7906, Train Acc: 72.22%, Val Loss: 0.8129, Val Acc: 70.86%
Epoch [13/40] - Train Loss: 0.7490, Train Acc: 73.81%, Val Loss: 0.8134, Val Acc: 71.47%
Epoch [14/40] - Train Loss: 0.7169, Train Acc: 74.81%, Val Loss: 0.7719, Val Acc: 72.37%
Epoch [15/40] - Train Loss: 0.6836, Train Acc: 76.15%, Val Loss: 0.8070, Val Acc: 71.84%
Epoch [16/40] - Train Loss: 0.6493, Train Acc: 77.22%, Val Loss: 0.7501, Val Acc: 73.35%
Epoch [17/40] - Train Loss: 0.6209, Train Acc: 78.27%, Val Loss: 0.7316, Val Acc: 74.25%
Epoch [18/40] - Train Loss: 0.5879, Train Acc: 79.51%, Val Loss: 0.7441, Val Acc: 73.89%
Epoch [19/40] - Train Loss: 0.5613, Train Acc: 80.50%, Val Loss: 0.7228, Val Acc: 75.04%
Epoch [20/40] - Train Loss: 0.5352, Train Acc: 81.38%, Val Loss: 0.7432, Val Acc: 74.61%
Epoch [21/40] - Train Loss: 0.5082, Train Acc: 82.18%, Val Loss: 0.7124, Val Acc: 75.55%
Epoch [22/40] - Train Loss: 0.4819, Train Acc: 83.24%, Val Loss: 0.7341, Val Acc: 74.53%
Epoch [23/40] - Train Loss: 0.4536, Train Acc: 84.19%, Val Loss: 0.6665, Val Acc: 77.21%
Epoch [24/40] - Train Loss: 0.4250, Train Acc: 85.13%, Val Loss: 0.7033, Val Acc: 76.02%
Epoch [25/40] - Train Loss: 0.4081, Train Acc: 85.86%, Val Loss: 0.6511, Val Acc: 78.06%
Epoch [26/40] - Train Loss: 0.3761, Train Acc: 86.90%, Val Loss: 0.6880, Val Acc: 77.18%
Epoch [27/40] - Train Loss: 0.3504, Train Acc: 87.88%, Val Loss: 0.6819, Val Acc: 77.21%
Epoch [28/40] - Train Loss: 0.3323, Train Acc: 88.33%, Val Loss: 0.6693, Val Acc: 78.09%
Epoch [29/40] - Train Loss: 0.3029, Train Acc: 89.32%, Val Loss: 0.7796, Val Acc: 76.36%
Epoch [30/40] - Train Loss: 0.2809, Train Acc: 90.26%, Val Loss: 0.7494, Val Acc: 77.02%
Epoch [31/40] - Train Loss: 0.2592, Train Acc: 91.16%, Val Loss: 0.7145, Val Acc: 77.90%
Epoch [32/40] - Train Loss: 0.2376, Train Acc: 91.82%, Val Loss: 0.7065, Val Acc: 78.25%
Epoch [33/40] - Train Loss: 0.2192, Train Acc: 92.49%, Val Loss: 0.7383, Val Acc: 78.35%
Epoch [34/40] - Train Loss: 0.1931, Train Acc: 93.42%, Val Loss: 0.8008, Val Acc: 76.83%
Epoch [35/40] - Train Loss: 0.1665, Train Acc: 94.44%, Val Loss: 0.8355, Val Acc: 77.48%
Epoch [36/40] - Train Loss: 0.1578, Train Acc: 94.83%, Val Loss: 0.8076, Val Acc: 78.13%
Epoch [37/40] - Train Loss: 0.1346, Train Acc: 95.61%, Val Loss: 0.8608, Val Acc: 77.76%
Epoch [38/40] - Train Loss: 0.1170, Train Acc: 96.19%, Val Loss: 0.9311, Val Acc: 77.46%
Epoch [39/40] - Train Loss: 0.1076, Train Acc: 96.57%, Val Loss: 0.8784, Val Acc: 77.78%
Epoch [40/40] - Train Loss: 0.0864, Train Acc: 97.37%, Val Loss: 0.9959, Val Acc: 76.57%

--- Running SGD with learning rate 0.001 ---
Epoch [1/40] - Train Loss: 2.2057, Train Acc: 17.62%, Val Loss: 2.0454, Val Acc: 25.44%
Epoch [2/40] - Train Loss: 1.9328, Train Acc: 28.72%, Val Loss: 1.8113, Val Acc: 33.74%
Epoch [3/40] - Train Loss: 1.7466, Train Acc: 35.05%, Val Loss: 1.6839, Val Acc: 36.30%
Epoch [4/40] - Train Loss: 1.6287, Train Acc: 39.39%, Val Loss: 1.5731, Val Acc: 41.56%
Epoch [5/40] - Train Loss: 1.5442, Train Acc: 43.18%, Val Loss: 1.5248, Val Acc: 43.69%
Epoch [6/40] - Train Loss: 1.4828, Train Acc: 45.66%, Val Loss: 1.4664, Val Acc: 45.97%
Epoch [7/40] - Train Loss: 1.4328, Train Acc: 47.79%, Val Loss: 1.4256, Val Acc: 47.76%
Epoch [8/40] - Train Loss: 1.3871, Train Acc: 49.77%, Val Loss: 1.3287, Val Acc: 52.11%
Epoch [9/40] - Train Loss: 1.3369, Train Acc: 51.98%, Val Loss: 1.3250, Val Acc: 52.01%
Epoch [10/40] - Train Loss: 1.2821, Train Acc: 54.17%, Val Loss: 1.2452, Val Acc: 55.84%
Epoch [11/40] - Train Loss: 1.2343, Train Acc: 55.75%, Val Loss: 1.2077, Val Acc: 56.60%
Epoch [12/40] - Train Loss: 1.1899, Train Acc: 57.71%, Val Loss: 1.1602, Val Acc: 58.63%
Epoch [13/40] - Train Loss: 1.1476, Train Acc: 59.26%, Val Loss: 1.1816, Val Acc: 58.58%
Epoch [14/40] - Train Loss: 1.1105, Train Acc: 60.89%, Val Loss: 1.0846, Val Acc: 61.48%
Epoch [15/40] - Train Loss: 1.0720, Train Acc: 62.45%, Val Loss: 1.0872, Val Acc: 62.37%
Epoch [16/40] - Train Loss: 1.0439, Train Acc: 63.21%, Val Loss: 1.0438, Val Acc: 62.96%
Epoch [17/40] - Train Loss: 1.0135, Train Acc: 64.20%, Val Loss: 1.0097, Val Acc: 64.71%
Epoch [18/40] - Train Loss: 0.9865, Train Acc: 65.29%, Val Loss: 0.9935, Val Acc: 64.71%
Epoch [19/40] - Train Loss: 0.9610, Train Acc: 66.14%, Val Loss: 0.9894, Val Acc: 65.52%
Epoch [20/40] - Train Loss: 0.9412, Train Acc: 66.93%, Val Loss: 0.9560, Val Acc: 66.31%
Epoch [21/40] - Train Loss: 0.9163, Train Acc: 67.80%, Val Loss: 0.9694, Val Acc: 65.98%
Epoch [22/40] - Train Loss: 0.8990, Train Acc: 68.36%, Val Loss: 0.9391, Val Acc: 66.46%
Epoch [23/40] - Train Loss: 0.8766, Train Acc: 69.31%, Val Loss: 0.9115, Val Acc: 67.79%
Epoch [24/40] - Train Loss: 0.8597, Train Acc: 69.71%, Val Loss: 0.9518, Val Acc: 65.88%
Epoch [25/40] - Train Loss: 0.8410, Train Acc: 70.48%, Val Loss: 0.8813, Val Acc: 68.60%
Epoch [26/40] - Train Loss: 0.8191, Train Acc: 71.25%, Val Loss: 0.8604, Val Acc: 69.41%
Epoch [27/40] - Train Loss: 0.8011, Train Acc: 71.93%, Val Loss: 0.8625, Val Acc: 69.50%
Epoch [28/40] - Train Loss: 0.7863, Train Acc: 72.35%, Val Loss: 0.8376, Val Acc: 70.43%
Epoch [29/40] - Train Loss: 0.7679, Train Acc: 73.19%, Val Loss: 0.8870, Val Acc: 68.60%
Epoch [30/40] - Train Loss: 0.7584, Train Acc: 73.39%, Val Loss: 0.8624, Val Acc: 70.08%
Epoch [31/40] - Train Loss: 0.7403, Train Acc: 74.04%, Val Loss: 0.8092, Val Acc: 71.47%
Epoch [32/40] - Train Loss: 0.7237, Train Acc: 74.64%, Val Loss: 0.7808, Val Acc: 71.80%
Epoch [33/40] - Train Loss: 0.7096, Train Acc: 75.39%, Val Loss: 0.7701, Val Acc: 72.86%
Epoch [34/40] - Train Loss: 0.6958, Train Acc: 75.54%, Val Loss: 0.7886, Val Acc: 72.28%
Epoch [35/40] - Train Loss: 0.6772, Train Acc: 76.46%, Val Loss: 0.7700, Val Acc: 72.85%
Epoch [36/40] - Train Loss: 0.6674, Train Acc: 76.71%, Val Loss: 0.7709, Val Acc: 72.70%
Epoch [37/40] - Train Loss: 0.6539, Train Acc: 77.31%, Val Loss: 0.7604, Val Acc: 73.25%
Epoch [38/40] - Train Loss: 0.6422, Train Acc: 77.58%, Val Loss: 0.7652, Val Acc: 73.63%
Epoch [39/40] - Train Loss: 0.6302, Train Acc: 77.93%, Val Loss: 0.7355, Val Acc: 74.37%
Epoch [40/40] - Train Loss: 0.6115, Train Acc: 78.85%, Val Loss: 0.7529, Val Acc: 73.57%

--- Running SGD with learning rate 0.0003 ---
Epoch [1/40] - Train Loss: 2.2935, Train Acc: 11.75%, Val Loss: 2.2764, Val Acc: 15.44%
Epoch [2/40] - Train Loss: 2.2404, Train Acc: 17.88%, Val Loss: 2.1796, Val Acc: 23.11%
Epoch [3/40] - Train Loss: 2.1101, Train Acc: 23.38%, Val Loss: 2.0507, Val Acc: 26.08%
Epoch [4/40] - Train Loss: 2.0172, Train Acc: 26.64%, Val Loss: 1.9737, Val Acc: 27.36%
Epoch [5/40] - Train Loss: 1.9458, Train Acc: 28.77%, Val Loss: 1.9039, Val Acc: 30.88%
Epoch [6/40] - Train Loss: 1.8758, Train Acc: 30.90%, Val Loss: 1.8314, Val Acc: 32.12%
Epoch [7/40] - Train Loss: 1.8060, Train Acc: 33.10%, Val Loss: 1.7700, Val Acc: 34.70%
Epoch [8/40] - Train Loss: 1.7484, Train Acc: 35.08%, Val Loss: 1.7091, Val Acc: 36.55%
Epoch [9/40] - Train Loss: 1.7032, Train Acc: 36.69%, Val Loss: 1.6678, Val Acc: 38.16%
Epoch [10/40] - Train Loss: 1.6581, Train Acc: 38.40%, Val Loss: 1.6321, Val Acc: 39.72%
Epoch [11/40] - Train Loss: 1.6169, Train Acc: 40.30%, Val Loss: 1.6120, Val Acc: 40.81%
Epoch [12/40] - Train Loss: 1.5796, Train Acc: 41.71%, Val Loss: 1.5480, Val Acc: 42.95%
Epoch [13/40] - Train Loss: 1.5456, Train Acc: 43.14%, Val Loss: 1.5236, Val Acc: 43.54%
Epoch [14/40] - Train Loss: 1.5246, Train Acc: 43.86%, Val Loss: 1.4937, Val Acc: 44.98%
Epoch [15/40] - Train Loss: 1.4970, Train Acc: 45.12%, Val Loss: 1.4659, Val Acc: 46.30%
Epoch [16/40] - Train Loss: 1.4769, Train Acc: 45.94%, Val Loss: 1.4485, Val Acc: 47.09%
Epoch [17/40] - Train Loss: 1.4595, Train Acc: 46.78%, Val Loss: 1.4602, Val Acc: 45.91%
Epoch [18/40] - Train Loss: 1.4422, Train Acc: 47.40%, Val Loss: 1.4287, Val Acc: 47.74%
Epoch [19/40] - Train Loss: 1.4236, Train Acc: 48.16%, Val Loss: 1.5113, Val Acc: 44.43%
Epoch [20/40] - Train Loss: 1.4124, Train Acc: 48.60%, Val Loss: 1.3882, Val Acc: 49.57%
Epoch [21/40] - Train Loss: 1.3925, Train Acc: 49.39%, Val Loss: 1.4192, Val Acc: 47.39%
Epoch [22/40] - Train Loss: 1.3774, Train Acc: 50.15%, Val Loss: 1.3932, Val Acc: 49.68%
Epoch [23/40] - Train Loss: 1.3599, Train Acc: 50.67%, Val Loss: 1.3519, Val Acc: 51.41%
Epoch [24/40] - Train Loss: 1.3457, Train Acc: 51.39%, Val Loss: 1.3382, Val Acc: 51.84%
Epoch [25/40] - Train Loss: 1.3281, Train Acc: 52.06%, Val Loss: 1.2995, Val Acc: 53.38%
Epoch [26/40] - Train Loss: 1.3110, Train Acc: 52.91%, Val Loss: 1.2899, Val Acc: 53.72%
Epoch [27/40] - Train Loss: 1.2943, Train Acc: 53.47%, Val Loss: 1.2955, Val Acc: 52.93%
Epoch [28/40] - Train Loss: 1.2794, Train Acc: 53.96%, Val Loss: 1.2819, Val Acc: 54.13%
Epoch [29/40] - Train Loss: 1.2615, Train Acc: 54.76%, Val Loss: 1.2581, Val Acc: 55.29%
Epoch [30/40] - Train Loss: 1.2429, Train Acc: 55.52%, Val Loss: 1.2408, Val Acc: 55.41%
Epoch [31/40] - Train Loss: 1.2282, Train Acc: 56.20%, Val Loss: 1.2684, Val Acc: 54.52%
Epoch [32/40] - Train Loss: 1.2115, Train Acc: 56.85%, Val Loss: 1.2144, Val Acc: 56.55%
Epoch [33/40] - Train Loss: 1.1999, Train Acc: 57.28%, Val Loss: 1.1930, Val Acc: 57.29%
Epoch [34/40] - Train Loss: 1.1806, Train Acc: 58.26%, Val Loss: 1.1784, Val Acc: 58.68%
Epoch [35/40] - Train Loss: 1.1671, Train Acc: 58.68%, Val Loss: 1.1649, Val Acc: 59.18%
Epoch [36/40] - Train Loss: 1.1515, Train Acc: 59.27%, Val Loss: 1.1391, Val Acc: 60.02%
Epoch [37/40] - Train Loss: 1.1310, Train Acc: 60.09%, Val Loss: 1.1316, Val Acc: 59.83%
Epoch [38/40] - Train Loss: 1.1247, Train Acc: 60.30%, Val Loss: 1.1305, Val Acc: 59.94%
Epoch [39/40] - Train Loss: 1.1082, Train Acc: 60.96%, Val Loss: 1.1024, Val Acc: 61.36%
Epoch [40/40] - Train Loss: 1.0962, Train Acc: 61.23%, Val Loss: 1.0987, Val Acc: 61.38%

================================================================================
SUMMARY TABLE - BEST VALIDATION ACCURACIES
================================================================================

Configuration                       Best Val Accuracy (%)
-------------------------------------------------------
No Augmentation                     79.37               
Data Aug 1 (Mild)                   81.40               
Data Aug 2 (Aggressive)             80.05               
Baseline + Dropout                  76.63               
Baseline + BatchNorm                76.12               
Baseline + BN + Dropout             77.14               
Zero Initialization                 10.00               
SGD lr=0.003                        78.35               
SGD lr=0.001                        74.37               
SGD lr=0.0003                       61.38               

Results saved to 'results_summary.txt'
All plots saved as PNG files
